# The K programming language ("Klang") - What, and Why?

### The beginning...

Klang started as Kragle - a system for building interactions between sites/services/apis. Simple "scripts" built from "standard" components, which abstracted away the necessary format and structure conversions for the network messages in question (eg: HTTP request/response, et al.) - I wanted a highly re-usable and homogenous way to connect disparate systems, regardless of their pedigree (protocol, OS, etc), and to automate those interactions.

This initial goal was implemented and incorporated into a working system (Kragle.io, although only ever released as a private beta), and as part of that implementation I derived a few keys pieces of (abstract) core functionality, which the rest was built upon and enabled thereby:

1. A pluggable system for supporting various kinds of network conversations, eg: http, dns, etc. In addition to that, each system would require different variants/configurations: http, https, https-with-basic, https-with-oauth, etc.

    1. By itself, this is far from innovative... but the main reason behind this requirement is to provide a *universal* interface to sending and receiving messages from a foreign system. Unlike a programming language, where you can import different libraries which provide interfaces to various protocols, the system itself is upgraded with support for a given protocol, and as long as you're passing the (homogenous) interface an instance of a data structure conforming to the relevant structure schema for the message/procotol/etc type you're trying to send to/receive from, any 'userland' code/configuration/specification needs to know *nothing* about the low-level details of the protocol/etc.
    
    ^^^ last sent too long, awk

1. Strict usage of a schema system of some sort for all structures exposed to userland, in order to support things like validation of incoming/outgoing messages, derivation of required fields in the input structure to an output operation, etc.

1. A convention for specifying data structures to be used as 'conversion templates', so that one system's message type can be converted into another dynamically and on the fly, optionally via configurable parameters for each instance of the conversion. ("Produce a structure of type A, using structure B as the template, with structure C as input.")

*(There were other requirements unique to the system, but which were specific to the system in it's incarnation as a service exposed as an api/black box system, where end-users would have no direct access to the code driving it... as such I'll attempt to strip/limit my references to such requirements, as they are not relevant to the justification of Klang.)*

### The epiphany...

Once Kragle had been fully implemented in terms of it's core engine, backend, and api (lacking only a frontend), I was then forced to finally deal with a nagging issue at the time: how to design the UI such that the concepts involved were intuitive enough for the end user (or even average "power user"/developer/etc).

While struggling with this issue, it dawned on me one day that the reason it's so hard to explain is that what I had essentially produced was a (albeit limited) form of a new programming language, not a specific service that would be sated by a one or two sentence description of 'what it does'. Like a programming language, the intent was that you build things with it, and that the things you built were limited only be the context and imagination of the user.

So... why not just make it a language instead?

### But is a whole new language really justified here?

Depends who you ask, and most (myself included, if I was someone else pitching me this idea) would probably have the immediate urge to answer "no, for fuck's sake, we have enough already!".

So I tried to convince myself that no, I'm delusional, you can't have enough good reasons to build a whole new language just to give your Kragle "pieces" a home.

However try as I might, this nagging realization that I needed to make it a *language* and not a service/api/etc of some kind persisted, try as I might to "make" it something else in my head.

### Maybe it's an operating system instead?

Maybe... but I don't think so. And I don't think it should be (although as you'll see me ultimately decide, a *virtual machine* is another story... and highly tied to 'interpreted' type language anyways, depending on how pendantic you want to be about the definition of interpreted language).

The main job of an operating system is to control and manage userland access to real system resources (disks, memory, network interfaces, processors, etc), to provide a standard (or at least consistent), abstracted interface to those resources, and to execute other programs (the OS is itself just a program).

The first two of those jobs is already done rather well by the existing options (well enough, anyways, depending on the option ;)), and the second as well, although the second is something we'll try to improve upon, as you'll see).

The last, at first glance, seems like it might actually be an attribute of a language, depending on your viewpoint - again, for *interpreted* language, the "language" is essentially a virtual machine that is 'executing' your source (program) on demand. We'll leave this point for now, to be addressed later on either explicitly or at least implicitly.

### Ok, so then what are the defining characteristics of a language, and how do the Kragle core features fit in?

From Wikipedia: "A programming language is a formal computer language or constructed language designed to communicate instructions to a machine, particularly a computer." Simple enough.

Let's examine our core Kragle pieces in the context of this definition:

* (Kragle feature): Homogenous interface for communicating with (*typically* remote) foreign systems.

    * We're saying we want a way of telling the computer to send or receive a given message to/from system X. That's pretty much spot-on - we're defining a "language" of sorts (the formal syntax of the mechanism in question) to be used for instructing the computer to communicate with "some system" on our behalf. Our particular incarnation of this definition with respect to the homogenous foreign communication is simply very high-level.
    
* (Kragle feature): Strict usage of data structure schema, for validating/etc "all the things". (Note here the recurring theme of homogeneity - we don't want to worry about boilerplate more than once - we just want to be able to send our messages with as little 'in the way' as possible.)

    * When communicating instructions to a machine, one major requirement is always going to be that the message you use for your communication is well-formed - computers are nothing if not precise and unforgiving. This is the root of the "issue" and never-ending discussion around Type Systems: which are best, whether or not they are necessary/desirable, etc. Type systems are intimately tied to the ability to speak to a machine (the goal of a programming language), and our 'structure schema' requirement is just another instance of this: we want a homogenous, high-level way of talking to arbitrary systems... so we also need/want a homogenous, high-level way of validating our messages (data).
    
* (Kragle feature): Ability to specify conversion templates, as arbitrary data structures, with another arbitrary structure as input to the template.

    * Admittedly, rather far removed from the definition of a programming language... but let's break it down a bit. Why do any languages other than machine code (or assembly) exist at all, if the goal is to simply communicate with the machine? The machine already has a 'native' language that we could theoretically use to speak with it. The obvious answer is that writing code in 0's and 1's would be a horrible existence for anyone involved with computers. Speaking a language of our own devising, and then having another program automatically convert this into something the machine will understand, is simply a better approach. Among many other reasons, this **allows us to encapsulate many basic, common operations into easier-to-understand things with names, like "if" or "function"**, and allows us write out program in such a way that we can refer to these extremely common operations by name, instead of implementing their mechanism every time we wish to use them. Converting data structures in this way (and associated operations) is a very, very common operation in the realm of integrating two heterogeneous protocols/systems/etc, and so having a 'native operation', as well as special syntax for that operation, can be viewed as another example of encapsulating the *most common* operations in your environment as core elements of a the language in which it is programmed, and the set of basic "operations" (or "features", etc) of a language are the differentiating factor (along with syntax), that separate one language from any other. In our case, we just have a few new/additional basic operations that are requirements for the type of programming we will be doing (integrating arbitrary, heterogenous foreign systems with each other).

### So then why not just some form of DSL and accompanying software? Or just a library? (This is not the right spot/heading for this???)

Ah... now we're getting closer to the point. (Or are we? I've lost my train of thought w.r.t. where I was going for at least 3 or 4 paragraphs now :/ - 2016-09-17)

### Why is syntax important? (Big part of the reason for 'full' language vs. DSL or library)

### Why is determinism important? (Portability... etc?)

### Why immutability? (WORM data backend, determinism, etc)

### Ok... so then what are our core requirements of a language (computation in general), from which all else is built?

To re-phrase that a little better, what's the difference between a computation/algorithm/etc of some kind, and a simple *operation*? 

The difference is less clear that one might thing. Even a simple operation like addition of two positive integers can itself be considered an algorithm if it's broken down into a step by step process. ("Take the first number, count UP from it by 1, and iterate X times, where X is the second number...", etc). It already gets more complicated internally if you allow negative integers in the "input domain" (ie: now you have a branching statement: "If the second number is negative, count DOWN instead...").

Perhaps the difference is that in order for something to be considered a calculation, it needs to be able to perform *useful* work.

Let's imagine as simple an example as we can:

* Values ("data") are limited to either TRUE or FALSE.
* We want an algorithm (program) for calculating the inverse of a value

One option would be to have a single function, the identity function, which just produces whatever you feed it).

If I have an "input" value, and I want to calculate the result, I could execute an identity function and pass it the opposite of my input, so that it returns that same value:

```
F identity(x) x
```

But this is far from useful - I'm still doing the 'calculation' myself (determining if the identity function needs to be passed TRUE or FALSE to get the opposite of what my input is) - the computer is essentially just encoding/formatting/outputing my values for me.

A *slightly* more useful program would be the following:

```
F negation(x) if (x) FALSE else TRUE
```

Still fairly trivial, but there's an important difference - the "work" has been moved out of my head, and into the program - the program is now making the relevant *decision* on my behalf, instead of just recording/echoing the data.

This decision-making ability is central to the idea of computation - the ability to modify internal behaviour based on varying inputs. The decision making ability is always modelled as a branching statement of some kind - some form of "if", and is typically a key component of ALL languages, including one-instruction-set computers and simple lisp-like languages like Scheme.

The ability to process input is also a central one. Without any input, a given algorithm will always produce the same results - ie: it will always do the same thing, with zero variability in the potential outcome.

In order to process input, we need some way of referring to it, and so in order to perform any useful work an algorithm also needs the ability to bind it's argument(s). (More generally, our language/computation machine/etc needs a way to store a value (the argument), and also a way to refer to it (a variable name, or an index into an infinite-length Turing ticker tape, etc etc).

Already, we have two core requirements to support *any* kind of *useful* computation whatsoever:

1. A branching ability (some form of "if", so we can make decisions in our algorithm)

1. The ability to store a value, and refer to it somehow

1. The ability to signify that the computation is finished, ie: an exit condition

Are there more? Let's talk about complexity for a moment.

Imaginge the bare minimum requirements we've been able to derive thus far, and describe a few assumptions that are required:

* All operations in our language will always refer to the stored value, if a value is needed
* Output is always to the single stored "place"
* When a program terminates, it's output is the value in the "place"

Here is a simple implementation of such a language (or virtual machine/interpreter, etc):

In [1]:

import sys


class HaltError(Exception):
    def __init__(self, *args):
        self.final_value = args[0]
        
        super(*args[1:])


# This is what implements the execution at each processing loop
def call_process_function(storage):
    """
    Execution phase of the language's processing step.
    
    storage - this is both our 'program' and our 'data'
    """
    
    if not isinstance(storage, bool):
        raise Exception('Klang only supports the value True or False!')
    
    if storage:
        # This defines the behaviour of our virtual machine for the instruction/value True
        return true_behaviour()
    else:
        # This defines the behaviour of our virtual machine for the instruction/value False
        return false_behaviour()


def run_program(storage):
    """This is our main processing loop - our virtual machine. We pass it our program, and it executes it."""

    # Process - loop forever (we always execute the current, and only, instruction)
    while True:
        
        try:
            storage = call_process_function(storage)
        
        # The "operator" functions called by call_process_function(..) will raise HaltError with the
        # value they with to halt after writing as HaltError.final_value...
        except (HaltError, he):
            storage = he.final_value
            
            print(storage)   # Output!
            
            sys.exit(0)   # Normal exit

What is the range of possible algorithms we could write in such an environment? Without a way to 'build up' a sequence of operations (where would you store it?), we're limited to the input to the 'program' (starting value in the single storage spot), and builtin operators. We are limited to two values, TRUE and FALSE, since so far those are the only values our language can represent in the example above, and since there is no other storage, the operation to be performed can only be dictated by the current value. In this situation, there are exactly two possible programs to be written:

1. The program "TRUE"
1. The program "FALSE"

*(A language with only **one** possible value is fairly useless analysis-wise, as it can only produce one value, itself.)*

Regardless of how you define the "TRUE" operator the input to it will always be the same ("TRUE"), and therefore no variability will result: the result will always be the same, it either:

1. results in FALSE (possibly infinite loop)
1. results in FALSE, and exits
1. results in TRUE (infinite loop - not useful)
1. results in TRUE, and exits

Which of these depends entirely on what the "TRUE" operator does internally (either returns TRUE or FALSE, loops forever inside the machine somewhere, or exits). So we can say that for a given implementation of our "binary language", the program TRUE will always have exactly one behaviour.

The "FALSE" program works exactly the same way in this scenario, namely it has exactly one behaviour, which depending on the implementation will has exactly one of these possible behaviours.

Here are some examples of implementations of call_process_function:

In [2]:
def sample_true_handler_1():
    """Writes a False value to the storage place, allows execution to continue."""
    return False

def sample_true_handler_2():
    """Writes nothing (writes itself, ie: True), allows execution to continue."""
    return True

def sample_true_handler_3():
    """Writes a False value to the storage place, halts execution."""
    raise HaltError(False, 'Processing complete.')
    
def sample_true_handler_4():
    """Writes nothing (writes itself, ie: True), halts execution."""
    raise HaltError(True, 'Processing complete.')

def sample_false_handler_1():
    """Writes a True value to the storage place, allows execution to continue."""
    return True

def sample_false_handler_2():
    """Writes nothing (writes itself, ie: False), allows execution to continue."""
    return False

def sample_false_handler_3():
    """Writes a True value to the storage place, halts execution."""
    raise HaltError(True, 'Processing complete.')
    
def sample_false_handler_4():
    """Writes nothing (writes itself, ie: False), halts execution."""
    raise HaltError(False, 'Processing complete.')

*NB: Our "branching" ability exists in the ability to have more than one value stored - since our stored value is also used as the operations, we "execute" different behaviour depending on the "input" (starting value)... hence, branching/decision making. If you specify that TRUE exits, and FALSE returns TRUE, then you have "If the value is true, exit, otherwise re-write the value to true."*

Essentially what we've done is implement some combinatorial logic: a truth table (unary, because we only have one 'place') whose input-output mapping corresponds to how we define the TRUE and FALSE operations.

Implementing a truth table syntax does not a language make.

So far, this is not very useful, we are still only at a 1:1 mapping of program to potential output, we're still basically encoding the desired output by providing the relevant input that will generate it, and the decision making is thus still in our heads, not in the program - I don't think we've defined ALL the requirements for a language yet. 

How can we further enhance the abilities of our machine to allow for more variety (infinite, ideally) in the programs we may wish to write? Also, what is the point - why don't we just create a language that does exactly one thing? Well, for starters that's a program/algorithm, not a langauge for *making* programs/algorithms :) Also, we should keep in mind the implied goal of any (good) language: strike a useful balance of syntax complexity vs. the projected frequency of the need to use that piece of syntax, so that ideas may be communicated effectively and efficiently (ie: vocabulary size vs. frequency of word use, for a human language). 

In the context of our theoretical language/machine that we've thus far been using to attempt to derive the most basic requirements for computation, there are a few easy to identify options:

* increase vocabulary (allow for more than two values in the language)
* increase storage (to allow for more than one value to be stored/examined)
* a combination of the two

Increased vocabulary means that we can have a different behaviour for every possible input value. We can grow this ability to infinity by using a theoretically infinite range of allowed values (say, the set of natural numbers). Now, for any given desired output, there could be a given input which would produce it. So, we can produce any desired output (within the range of the allowed values in the language). But, we STILL just have a 1:1 mapping between behaviour (output/result) and input, so we would *still* need to perform the 'calculation' in our head and then specify the input (program) which would correspond to the output we want. Still not useful.

*This last example (above) of infinite vocabulary but only a single 'storage place' amounts to building a **machine**, not a **computer**... a machine takes a very specific input and does a single specific job, and produces some form of output... so we can build any machine we want with a language of infinite vocabulary... but we need to build a new machine for each operation.*

Increased storage poses a more interesting new scenario. Now we have the concept of an instruction pointer - which 'cell' in the data do we use as the 'next value to be processed'. This adds complexity to the ranges of potential inputs and outputs. If the amount of storage is assumed to extend indefinitely, the range of potential inputs (programs we can write) again becomes infinite. Let's try to analyze what this means in terms of useful programs we could write:

* We still only have two options for the 'current' operation: TRUE or FALSE

* "Moving" (ie: the current instruction pointer) is now one of the possible internal operations that can be performed as part of the execution of a given instruction (where instructions are still just either TRUE or FALSE)

* But we can define a sequence of operations, by adding instruction pointer changes (movement through memory) to the base behaviour of an operator: it can now perform the conbinatorial logic (unary truth table), and then move either right or left depending on the (original) value. Which means that we have a (very limited) form of sequencing now, which we can use to control the flow of operations in a program. And also, because of the infinite storage, we can build programs of arbitrary length.

* So our total bag of tricks in terms of what we can "do" inside an "operation" is:
    * write TRUE to the current storage place
    * write FALSE to the current storage place
    * move the instruction pointer a pre-defined number of steps in either 'direction' (assuming a one-dimensional, infinite storage array, the simplest we can conceive).
    * a value could be empty (uninitialized, ie: null, if we move to a storage location that is not "occupied"), which means that we need a third possible "value" allowed in our 'language' 

* Let's specify that NULL is not a third allowed value, but simply signifies end-of-program when encountered, for the sake of simplicity

* Let's make up some arbitrary movement rules for the "TRUE" and "FALSE" operations:
    * TRUE moves the pointer moves three spots to the right
    * FALSE moves the pointer moves one spot to the right
    * NULL moves one spot to the left and ends execution (result is the value at the resulting place)
    
* Can we build any useful programs with this? If we represent a program's code (the storage "cells") as some kind of an array, we can now represent programs like this:

```
[ TRUE, NULL, TRUE, NULL ]   # if FALSE is prepended, this produces TRUE
[ NULL, FALSE, NULL ]   # if TRUE is prepended, this produces FALSE
```

* Assuming we pre-pend the "input" to our program, (ie: so the first example becomes ```[ TRUE, TRUE, NULL, TRUE, NULL ]``` for evaluating TRUE, etc), then can we modify these programs such that they're consolidated into a single program for evaluating the truth of the "input" thus defined?

```
[ NULL, NULL, TRUE, NULL, FALSE, NULL ]   # if TRUE is prepended, this produces FALSE
```

* This is another program that will properly calculate the "not" of TRUE, but it has the added characteristic of being compatible with the first program for calculating the "not" of FALSE. This is because the FALSE program depends on a TRUE being in position 1 (starting from 0, with our prepended input being the 0th element), a NULL being in position 4, and a TRUE being in position 3. Similarly, the new program for the "not" of TRUE depends upon a TRUE being in position 3, a NULL being in position 6, and a FALSE being in position 5.

* So because their dependencies don't conflict (ie: one doesn't need a TRUE in position X while the other needs a FALSE there), we can combine them into a new "program":

```
[ TRUE, NULL, TRUE, NULL, FALSE, NULL ]
```

* When prepending *either* a TRUE or FALSE to this program, it will properly calculate the opposite of it's input! We have moved a decision-making process from our heads, into a very basic program!

* Once the program completes, the value at the current pointer (FALSE, if our input is TRUE) no longer means "move right one spot" but rather "the result is FALSE" - we have combined logic and data in a single element (our TRUE and FALSE values)... a very necessary thing if we ever want a program to mean anything more than it's text, or a 1:1 mapping of that text to a result (ie: data can't JUST be data, but needs to exist in some form as logic as well - we need to be able to perform a specific action based on the value of some piece of data, and then use another piece of data as the result).

* We have also done a few other sneaky things: we are adding semantics other than "stop processing" to the NULL character, so we are now technically using a third "operator" (although our "operators" for the most part are still both code and data). We are also assuming that our 'program' can consist of any sequence of allowed values, **including NULL**, and that we can have a NULL positioned between two non-NULLs. Which seems reasonable, given our existing allowance of infinite one-dimensional storage. Neither of these are new requirements, however - they both fit implicitly into the requirement of an infinite chunk of one-dimensional storage, we simply dictate that any position without a value equates to NULL.

* What other kinds of programs can we build with just these rules? Can we build **binary** logic gates?

```
[ FALSE, FALSE,    NULL ]                               # OR, with two FALSEs as input
[ FALSE, TRUE,     NULL,  TRUE,  NULL ]                 # OR, with FALSE and TRUE as input (in that order)
[ TRUE,  FALSE,    NULL,  TRUE,  NULL,  TRUE,  NULL ]   # OR, with TRUE and FALSE as input
[ TRUE,  TRUE,     NULL,  TRUE,  NULL,  TRUE,  NULL ]   # OR, with two TRUEs as input

[ NULL,  TRUE,  NULL,  TRUE,  NULL ]   # Full program for calculating any input to OR (input to be prepended)
```

* We can!    Nix this bit: "However an interesting pattern emerges already: with our static "jumps" to another point in the program, it appears that the *size* of our program may be related to the vocabulary ("alphabet" of the language) and the number of inputs. For a one-memory-cell machine, we had two symbols in our alphabet (TRUE and FALSE), and one input (the program symbol) - this resulted in the ability to perform our computation..."

* The ability to jump to other points in our program can be thought of as the ability to create a tree structure in the code which we can follow, making decisions at each step. But these decisions are still only based on the instruction at that point in the code - other than building huge decision trees, we still have no way of knowing what a previous value was, aside from the fact that we're in a given 'branch' of the tree. But this means that there is at least a linear (if not worse) relation between the number of potential edges in our binary tree - we have to explicitly code the decision tree for every possible execution path. Not ideal.

* But let's keep exploring the bare-minimum language some more. Is there a maximum complexity we can determine? Ie: a limit to the size of a decision tree that we can represent with the inputs? Is there a limit to the number of inputs? Let's tackle inputs first...

* Let's attempt to identify a situation where our language will fail to serve us, and then try to understand why... what if we wanted to build a trinary logic gate, where the output should be TRUE if *only* one of the three inputs is true - a form of trinary exclusive-or. We will quickly find that it is impossible, since in a situation where we *need* to examine all of the inputs, we will always have a situation where certain input combinations (a TRUE in either the first or second position) will result in a skip ahead past one of the remaining inputs. Even if we reduce our jump distance to be 2 instead of 3, we will still have to skip the second input. And because a jump is always to the right, there's no way to go 'back' to a previous value.

* So it becomes obvious that in any sequential language (an algorithm is, by definition, always going to be sequential in some context), we require the ability to make a backwards jump in order to re-examine previous values.

* Let's add this ability as minimally as possible: instead of TRUE and FALSE, we will still have only two "values" to be represented (still TRUE and FALSE), but we will represent them with natural numbers, and specify that a zero indicates False, and anything else (positive integers) indicate TRUE. But now that we have more than two input representations, we can use the set of TRUE inputs (positive integers - an infinite set) in the following way: instead of jumping ahead 3 'spaces', the non-zero TRUE value indicates the absolute position in the program to jump to, when program positions (instruction indices) are indexed starting at 1 (because 0 is FALSE, we can't 0-index).

* Great, so now we could write a program like this, which returns to it's arguments even after a jump from argument 1:

```
[ 5, 7, 9,   0, 2,   0, 3,   0, NULL ]
```

* This program will follow this path (1-indexed): 1, 5, 2, 7, 3, 9, 8 (return value). So now we can go 'back' and re-process our arguments. We still have a problem however - it's easy to see that regardless of what preceded it, once the program arrives at a given instruction, the subsequent path will always be the same regardless of what preceded it. So we can *visit* "skipped" values, but we still can't make decisions based on prior values. What's the reason for this (besides "it's obvious")?

* The missing piece is the ability for the program to modify itself - ie: write data as well as read it. Being able to modify it's own code results in potentially different behaviour between executions of the same piece of code, depending on prior values... or *state*.

* Let's define as simple a 'write' rules as we can, sticking with the 'theme' of binary values as much as we can: Instead of simply jumping if non-zero, we *decrement* the current value by one before jumping. In the context of that new rule change, examine the following "code":

```
  Inputs.
  |  |  |
[ 7, 7, 7,   0, NULL, 0, 3,   11, 1, NULL ]

[ 0, NULL,                                    1, NULL ]
```

* The above code will only return TRUE if all three inputs are TRUE.

```
  Inputs......   # For this program, the value to use for TRUE is (<1-idx'ed input pos> * 2) + 5
  |  |   |   |
[ 7, 0, 11, 13,    0, NULL     2, 0, 0, NULL, 4, 0, 15, 1, NULL, 18, 1, NULL ]
```

* This is a simple pattern matcher... the pattern we want to match on (return TRUE) is encoded as follows:
    * if the last value should be FALSE:
        * set the command in pos 5 to be 16
        * set the command in pos 14 to be 0
    * otherwise, leave these "last input" commands as-is
    * for the rest of the inputs, set the commands as follows:
        * if input n should be TRUE, set command at pos (n * 2) + 5, to be n + 1, and set (n * 2) + 6 to be 0
        * if input n should be FALSE, set command at pos (n * 2) + 5, to be0, and set (n * 2) + 6 to be NULL

* What about loops?

```
 Input
   |
[ 15,    NULL, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4 ]
```

* This program subtracts 3 from it's input, as long as it's input is between 4 and 30 (otherwise it will not perform the calculation, or will never halt, respectively). The result, regardless of a TRUE or FALSE answer, is left in position 1.

* Can we make an infinite loop? No! Since values always decrease, and the value is also the command (jump to x on non-zero), combined with the fact that we can't jump 'left' of position 1 (remember, we're 1-indexing our positions), any potential program will eventually terminate.

* Can we "set a value", ie: cause a specific position to have an arbitrary value? No. The maximum value for any position is limited by it's initial value, meaning that if there's a "5" at a given position, that position can only ever fall between 0 and 5, inclusive. So if the max value in a whole program is '15', then regardless of any other values that program would never be able to produce a value greater than 15.

* More interestingly, once a position's value is 0, it can never increase again.

* And lastly, we might notice at this point that a given piece of code is entirely dependent upon it's position in the overall program... which means that we may figure out some clever way of combining small 'programs' in order to combine them into more complex programs, but this *also* means that any time we *move* (or even *change* in some cases) a part of the program, at the very least all the code following it will also need to change, if not code prior as well. Not very useful.

* Part (all?) of the problem seems to be that our *data* is tied to our *code* - they are one and the same. If we want to perform a jump to position X, from position Y, the value at position Y needs to first be X, and correspondingly if we want to write the value A into position B, the value at position B first needs to be A + 1, *and* that means we need to perform a jump to A + 1 in order to set the value A into position B. This leads to all sorta of acrobatics in the code, and the aforementioned limits (which are likely a subset of the full limitations inherent in the current approach).

* Another way to explain this is that because the jump is always to the current value, and we can only modify the current value... if we derive a way of setting a variable at position A to x, then that will always result in a jump to position x. Which also means that *every time we need to write the value 'x' somewhere, we need to then jump to position x, regardless of whether two separate pieces of code writing the value 'x' have anything to do with each other...* there will always be this hard-to-work-around restriction in program flow, and the resulting need to re-compute the values for the rest of the program every time a new piece of code is added or removed, at *best*.

* Let's presume an intellectual jump at this point, and realize that *instructions* and *values* need to be treated separately. This means two things:

    * The ability to write a result to an arbitrary position instead of to the 'current' position.
    
        * This is necessary because if we always write to the current position, then our data (the result) shares it's *address* with the instruction that produced it, and overwrites the instruction, hence the two would be coupled.
        
    * The ability to jump to a position other than the position indicated by the value of the result or the position of the result itself.
    
        * We need to be able to jump arbitrarily to a given position, regardless of the result of the current instruction.
        
* Also keep in mind that the reason for a jump is because we want to encode the decision-making process into the program, so that it can make decisions based on a value... and we've now said that we want to separate data from the operation, so we need to make the 'jump if' decision based on a "value" position, not part of the instruction!
        
* So let's change our 'machine' again:
    * values are now the set of integers (.., -1, 0, 1, ..), or NULL to trigger halt (in first argument position)
        * All integers (including negatives) allows us to produce values higher than the highest values in the original source, ie: when coupled with subtraction (subtracting a negative will increase a value)
    * We are still subtracting, only we will allow arbitrary subtractions instead of just decrement-by-one, so that we can increase our range of possible stored values in the program to encompass all integers, and also more importantly so that we can *increase* values (subtract negative) instead of always decrease!
    * Our decision to jump is still based on a zero/non-zero value, except instead of the zero being part of the operation (subtract 0), we examine the *result* of the operation for 0 to decide if a jump is necessary, that way the values in the subtraction can be whatever we require for the task at hand, and not 'anything but zero'.
    * Therefore, we need two values for our instruction: the positions in the program (addresses) holding the values to subtract (we will leave the result as the new value of one of these addresses, for now, and see if that's sufficient).
    * We also need a third value to use as part of the instruction: the address to jump to, if the result is non-zero.
    * We still just have a single instruction, which is a conditional jump (jump to address X if predicate), it simply spans three values (program addresses) - the single instruction now takes 3 inputs instead of one (we'll say the current instruction pointer and the following two in ascending address order, ie: instruction pointer is position 1, then the inputs for that instruction are at address 1, 2, and 3).
        
* In summary, our single-instruction machine now looks like this: For a given sequence of values, each of which is an integer or NULL, start at position 1, and halt if NULL. Otherwise, evaluate INSTRUCTION(A, B, C) where A, B, and C are the values at positions 1, 2, and 3, respectively. The INSTRUCTION to execute is "subtract A from B, store the result in B, and jump to C if the result is non-zero, otherwise proceed to instruction (current position + 3)".

* It turns out that this final modification to our bare-bones computing machine **is** Turing-complete... it is almost exactly the same machine/language as SUBLEQ (a one-instruction set computer, or OISC), which uses a "subtract and branch if result is non-positive (Less-than-or-EQ-to-zero)". In fact, it's even closer to the known OISC "SBNZ" ("Subtract and Branch if Not equal to Zero") - the only difference is SBNZ uses four arguments instead of three, for the sake of convenience (this allows many operations to be collapsed into a single instruction, instead of multiple instructions as we'll see below in the first example for our own new machines). Let's see what we can do with our new language/machine, to try and demonstrate this Turing-completeness and the suitability as a *practical, and useful*, bare-bones computing model. We'll use a new syntax, to make the sequence of instructions more obvious, as well as the addresses:

```
 -5:   17,    0,    1,   # input, uncond jump usage, uncond jump usage (etc, ie: decr)
 -2:   -1,   -4, NULL,   # incr usage, reset loop

                         # Begin "minus 4" function...
                         # Local variable "declarations"...
  1:    0,    0,    0,   # arg, return address, return value
  4:    0, NULL, NULL,   # counter
  
  7:   -4,   -3,   55,   # JUMP AHEAD TO START

                         # Entry point for a call to "minus 3" function...
 10:    7,    7,   13,   # zero counter, proceed to 13 unconditionally
 13:   -1,    7,   16,   # Subtract our 'reset value' from position 7, proceed to 16 unconditionally

                         # Main loop...
 16:   -2,    4,   19,   # increment argument (negative input), proceed to 13 unconditionally
 19:   -3,    7,   16,   # decrement counter, loop to 16 unless zero

                         # Write return value...
 22:   40,   40, NULL,   # Zero the return-zeroing first arg
 25:   41,   41, NULL,   # Zero the return-zeroing write dest (second arg)
 28:   44,   44, NULL,   # Zero the return-writing write dest (second arg)
 31:    6,   40,   34,   # Write arg1 for return-zeroing write
 34:    6,   41,   37,   # Write arg2 for return-zeroing write
 37:    6,   44,   40,   # Write arg2 for the return-writing write
 40:    0,    0, NULL,   # Zero the return value
 43:    4,    0,   46,   # Write the return value, proceed to 43 unconditionally

                         # Perform the return jump...
 46:   54,   54, NULL,   # Zero the return address
 49:    5,   54,   52,   # Write the return address
 52:   -4,   -3,    0,   # Jump to return address
                         # End "minus 4" function...

                         # Begin call-function (minus 4)
 55:    4,    4, NULL,   # Zero arg in function frame
 58:   -5,    4,   61,   # Write argument to function frame
 61:    6,    6, NULL,   # Zero return value address in function frame
 64:   77,    6,   67,   # Write return value address
 67:    5,    5, NULL,   # Zero return address address in function frame
 70:   76,    5,   73,   # Write return address address
 73:   -4,   -3,   10,   # Call function (jump)
                         # End call-function

 76:   79,   80, NULL,   # return address, return value
 
 79: NULL,    0, NULL,   # End program - halt
```

* This is significantly longer and more complex than the prior examples for previous "generations" of our machine, so let's break it down piece by piece:

```
 -5:   17,    0,    1,   # input, uncond jump usage, uncond jump usage (etc, ie: decr)
 -2:   -1,   -4, NULL,   # incr usage, reset loop
```

* This sets up our input (-5, presumably written by the 'interpreter' before execution begins), as well as some constants to utilize in various subsequent locations: 0, 1, -1, -4 (a loop reset value)

```
                         # Begin "minus 4" function...
                         # Local variable "declarations"...
  1:    0,    0,    0,   # arg, return address, return value
  4:    0, NULL, NULL,   # counter
```

* Here we begin a function definition - it starts with simply a bunch of values (addresses, really) to use as local variables in the function (counter), as well as spots for the 'caller' to fill in the argument to the function, the address to write the return value into, and the address to return execution to after the function completes.

```
  7:   -4,   -3,   55,   # JUMP AHEAD TO START
```

* This is our first instruction - the start of our program - and simply jumps ahead to the first instruction after our "function" definition - the code between here and the jump-to point is the body of the function - we want it available, but we don't want to execute it yet because we need to explicitly pass it a value, not use the default value in the argument address.

```
                         # Entry point for a call to "minus 3" function...
 10:    7,    7,   13,   # zero counter, proceed to 13 unconditionally
 13:   -1,    7,   16,   # Subtract our 'reset value' from position 7, proceed to 16 unconditionally
```

* Once we setup our local variables, we being the 'body' of our function by first zero'ing out the current value in the counter address by subtracting it from itself.

```
                         # Main loop...
 16:   -2,    4,   19,   # increment argument (negative input), proceed to 13 unconditionally
 19:   -3,    7,   16,   # decrement counter, loop to 16 unless zero
```

* This is the main loop of the function - decrement the argument 

```
                         # Write return value...
 22:   40,   40, NULL,   # Zero the return-zeroing first arg
 25:   41,   41, NULL,   # Zero the return-zeroing write dest (second arg)
 28:   44,   44, NULL,   # Zero the return-writing write dest (second arg)
 31:    6,   40,   34,   # Write arg1 for return-zeroing write
 34:    6,   41,   37,   # Write arg2 for return-zeroing write
 37:    6,   44,   40,   # Write arg2 for the return-writing write
 40:    0,    0, NULL,   # Zero the return value
 43:    4,    0,   46,   # Write the return value, proceed to 46 unconditionally
```

* This is the code which writes the return value, to the address specified by the caller (stored in the function's "frame"). This is easiest to understand in reverse: we end with the instruction which writes the value into the return address. To write an arbitrary value, we need to zero it first, which is the second last instruction here. We also need the address itself that is to be written... the next-last instructions (31 through 37) set the address in the final two instructions that write the value. These address-setting instructions are again an instance of 'set address X to an arbitrary value Y', so they need to be zeroed first as well, which is what the first three instructions in this block (22 through 38) do.

* We will soon introduce a "macro" into our syntax for writing code to simplify this repetetive chunk of code as it's used quite a bit.

```
                         # Perform the return jump...
 46:   54,   54, NULL,   # Zero the return address
 49:    5,   54,   52,   # Write the return address
 52:   -4,   -3,    0,   # Jump to return address
                         # End "minus 4" function...
```

* Here we finish the 'minus 4' function by returning to the address specified by the caller - once again we zero the address, then set it arbitrarily to what was passed by the caller, then perform the jump itself once the address is set.

```
                         # Begin call-function (minus 4)
 55:    4,    4, NULL,   # Zero arg in function frame
 58:   -5,    4,   61,   # Write argument to function frame
 61:    6,    6, NULL,   # Zero return value address in function frame
 64:   77,    6,   67,   # Write return value address
 67:    5,    5, NULL,   # Zero return address address in function frame
 70:   76,    5,   73,   # Write return address address
 73:   -4,   -3,   10,   # Call function (jump)
                         # End call-function
```

* Here again we see the same zero-then-write behaviour - to "call" our function, we write three values and then jump. The values to be written are the argument to the function (here copied from the user input at the beginning of the program), the address into which the result should be written (80, or the contents of address 77), and the address containing the next instruction to jump to on return from the function (79, or the contents fo address 76). We then jump unconditionally to address 10, which is the beginning of our "minus 4" function.

```
 76:   79,   80, NULL,   # return address, return value
```

* These lines simply hold two addresses to be used above in our function call...

```
 
 79: NULL,    0, NULL,   # End program - halt
```

* The end of our program - once the function call started at 55 returns, the result of "user input minus 4" will be present at address 80, and the function returns execution control to instruction 79, which is a NULL, and so the program halts.

* There's a lot of stuff going on in that program, despite it's high-level simplicity:

    * At the beginning, we are setting aside (allocating) space in 'memory', by initializing values at the beginning before the index 1 in the code where the program will begin execution - ie: allocating space for variable storage, as opposed to instruction storage, at addresses outside those where we will execute. We can think of this as declaring variables and constants.
    
    * We also allocate variables for use locally within the function we're about to declare...
    
    * We are then jumping past a big chunk of code, which we want to keep available but not executed yet (ie: the function body declaration).
    
    * Once we have defined this function, we can start to execute the main body of our program - namely, **pass** the input from the user as an argument to the "minus 4" function, and then exit with the result in our pre-selected 'output' position.
    
    * Another thing to notice is that we're writing to addresses directly, and a write-to-address has the ability to write *any* address, ie: including addresses holding code, as well as 'variable' addresses... and in the case of the latter, we are able to write to both "local" variables for the function, as well as "global" variables outside the function.
    
    * Lastly, the fact that we're defining a re-usable function period is important - this is enabled at least in part by our ability to write arbitrary values to arbitrary addresses (variables can be re-initialized), and our ability to jump to arbitrary locations from any other location - we can "enter" the function and then "return" when we're done, and this can be done from any point in the calling code.

* Let's build an actual SBNZ (our 3-operand version) interpreter!

In [3]:

###
### INTERPRETER STARTS HERE!
###


def grow_array_to_idx(arr, idx):
    while len(arr) < (idx + 1):
        arr.append(None)
        
    
def read_idx_or_none(arr, idx):
    grow_array_to_idx(arr, idx)
    
    return arr[idx]


def write_idx_or_none(arr, idx, val):
    grow_array_to_idx(arr, idx)
    
    arr[idx] = val
    
    return arr[idx]


def run_program(program, ip=0):
    """
    Run the program.
    
    program - an array of integers (our 'source code')
    ip - instruction pointers (defaults to 0, ie: first instruction (3-values) in the array)
    """
    
    # Validate that program is in fact an array
    
    if type(program) != type([]):
        raise Exception('SBNZ (Klang) programs must be passed as arrays!')
        
    program = list(program)   # Force a copy of the source, ie: do not modify source while executing!

    op_1_address = read_idx_or_none(program, ip)   # Read the next instruction's first operand
    
    # op_1_address, ie: value at the instruction pointer, is an address
    # pointing to the 2nd subtraction operand
    
    while op_1_address is not None:
        
        # operand 2 is an address of the value to use as first subtraction operand, and
        # operand 3 is the address to jump to if non-zero... so both of these have to be
        # non-null, ie: non-None (or at least operand 2 if the result is going to be zero).
        
        op_2_address = read_idx_or_none(program, ip + 1)
        op_3_address = read_idx_or_none(program, ip + 2)
        
        if op_2_address is None:
            raise Exception('SBNZ (Klang) requires operand 2 to be a valid address!')
        
        # the values "pointed" to by op_1_address and op_2_address should be integers (not None), and
        # so we validate this as well (non-null) and then cast to integer
        
        op_1_value = read_idx_or_none(program, op_1_address)
        op_2_value = read_idx_or_none(program, op_2_address)
        
        if op_1_value is None or op_2_value is None:
            raise Exception('SBNZ (Klang) requires each instruction to have 3 valid integer operands!')
        
        op_1_value = int(op_1_value)
        op_2_value = int(op_2_value)
        
        # Perform subtraction (2nd operand - 1st operand)
        
        result = op_2_value - op_1_value
        
        # Store result in the address pointed to by 2nd operand
        
        write_idx_or_none(program, op_2_address, result)
        
        if result == 0:   # Subtraction result is 0 - do not jump
            ip += 3
        else:
            if op_3_address is None:
                raise Exception('SBNZ (Klang) requires a valid address for operand 3, where a jump occurs.')
            
            ip = op_3_address   # Jump to instruction beginning at op_3_address
            
        # Prep for next iteration - read the new op_1_address
        
        op_1_address = read_idx_or_none(program, ip)

    # The result of the program, once we hit a None for operand 1 of the next instruction, is the
    # value at current-instruction-pointer-minus-one
    
    return read_idx_or_none(program, ip - 1)
    

In [5]:
seventeen_minus_4 = [
#  0:    
   -4,   -3,   54,   # JUMP AHEAD TO START
    
                     # Begin "minus 4" function...

                     # Entry point for a call to "minus 3" function...
#  3:
   51,   51,    6,   # zero counter, proceed to 6 unconditionally
   -1,   51,    9,   # Subtract our 'reset value' from counter, proceed to 9 unconditionally

                     # Main loop...
#  9:
   -2,   48,   12,   # increment argument (negative input), proceed to 12 unconditionally
   -3,   51,    9,   # decrement counter, loop to 9 unless zero

                     # Write return value...
# 15:
   33,   33, None,   # Zero the return-zeroing first arg
   34,   34, None,   # Zero the return-zeroing write dest (second arg)
   37,   37, None,   # Zero the return-writing write dest (second arg)
   50,   33,   27,   # Write arg1 for return-zeroing write
   50,   34,   30,   # Write arg2 for return-zeroing write
   50,   37,   33,   # Write arg2 for the return-writing write
# 33:
    0,    0, None,   # Zero the return value (return-zeroing write)
   48,    0,   39,   # Write the return value, proceed to 39 unconditionally (return-writing write)

                     # Perform the return jump...
# 39:
   47,   47, None,   # Zero the return address
   49,   47,   45,   # Write the return address
   -4,   -3,    0,   # Jump to return address
    
                     # Storage for "minus 4" variables (local variable "declarations")...
# 48:
    0,    0,    0,   # arg, return address, return value
    0, None, None,   # counter (, unused, unused)
    
                     # End "minus 4" function...

                     # Begin call-function (minus 4)
# 54:    
   48,   48, None,   # Zero arg in function frame
   -5,   48,   60,   # Write argument to function frame
   50,   50, None,   # Zero return value address in function frame
   76,   50,   66,   # Write return value address
# 66:
   49,   49, None,   # Zero return address address in function frame
   75,   49,   72,   # Write return address address
   -4,   -3,    3,   # Call function (jump)
    
                     # End call-function
# 75:
   78,   77,    0,   # return address, return value address, return value
    
# 78:
 None, None, None,   # End program - halt (, unused, unused)

                     # (global) variables and constants - accessed via idx -1, -2, etc
                     # (program input is at -5, ie: in this case our program gets 17)
# 81:
   17,    0,    1,   # input, uncond jump usage, uncond jump usage (etc, ie: decr)
   -1,   -4,         # incr usage, reset loop
]

run_program(seventeen_minus_4)

13

Let's take some of the code in the above example, and try to build some re-usable patterns:

```
# THE VARIABLE STUFF BELOW NEEDS TO BE RE-WRITTEN, IT'S ALREADY SLIGHTLY DEPRECATED!

# Variable - composed of four values:
# - type
#    0 == integer
#    1 == array
#    2 == function (see below)
#   -1 == array member (integer)
#   -2 == array member (nested array)
# - name
#   - for string names (arrays), the array itself has no name (this value is 0)
#   - for array elements, the name is an integer (index in the array)
#   - for arrays, the name is an address pointing to the name of the array variable
# - value (an address or an integer value)
#   - for integers, this is the value
#   - for arrays, this holds the address of the first element in the array
#   - for array members, this is the address holding the value of the given element (integer) or the address
#     of the element (array)
# - next (address of the next variable in the stack)
#   - for integers or arrays, this is next variable in the 'current' stack (or zero for no more variables)
#   - for array members, this holds the address of the next member (or zero for no more members)

# Having all variables take up the same number of 'spots' in our global program memory allows us to
# 'grow' the stack with new variables/array elements/etc, and have a reliable way of starting at 'the end'
# (via negative array indexing) to "walk" through the variables until we find what we need.

# Regular variables are located by walking the stack backwards from the end, through the linked list structure
# implemented by the variables' "next" value. Arrays work the other way around - you 'find' the 'root' array
# like any other variable, by walking backwards, but then the linked list of the array itself effectively then
# walks forward (address-wise) through the links until it finds the (possibly nested) value we're after.

# This may seem inefficient (any variable access will require multiple steps just to find it's address, etc), but
# we just need/want a *working* system right now, and being able to name variables and
# create (nested) arrays seems like a good-enough piece of functionality for now. Also, this nested linked-list
# structure has some useful qualities:
# - it allows us to just need a single value (the index of the end of our storage, ie: -1), to be able to locate
#   any value
# - we can easily 'allocate' storage by just reading an index X places beyond the current end (so that it's
#   back-filled with None), and then update our 'end' value if we're using one.
# - we can also store the positive end-index *in* the end position itself, and easily 'insert' new values into
#   the penultimate position via a simple set of steps, so that the 'end position' variable is always actually
#   in the end position:
#
#   1) add four to the value of the end position, and write this new value as an integer to the end of the stack,
#      pointing to it's own last address
#   2) write the new variable to the addresses that previously held the end positions, but leave the 'next' value
#      alone (ie: just write value, type, and name)

# Parser + internal representation/mechanisms: going from code to 'assembly'...
#
# - An integer literal in the code is converted into an anonymous integer on the global
#   stack at compile time
#   - the return of this in the assembly is an address on the stack of whatever references
#     the literal, which points to the (anonymous) variable on the stack
#
#    int :   <addr>,   <value>,      0,      0,   # next, value, name, type
#
#  where:
#    - addr: the address of the previous variable entry in the global namespace
#            * Does not necessarily have a name!
#    - value: the integer value (not an address)
#    - 0 (name): this integer is anonymous, ie: a program literal
#    - 0 (type): this is an integer
#
# - A string (or symbol) is converted into an anonymous array on the stack at compile time
#   - the return of this in the assembly is an address on the stack of the scope that
#     references the string
#
#    str :   <addr>,    str+4,      0,      1,   # next, value, name, type
#  str+4 :    str+8,   str+12,      0,     -1,   # first array member (anonymous/indexed)
#  str+8 :   str+12,   str+16,      0,     -1,   # second array member
# str+12 :      str,       65,      0,      0,   # first array member value (integer, 'A')
# str+16 :   str+12,       66,      0,      0,   # second array member value (integer, 'B')
#
#  where:
#    - addr: the address of the previous variable entry in the global namespace, before
#      the variables comprising this "array" are created
#
#   - the top of the current namespace now points to str+19
#     
# - An assignment (ie: binding) is converted into code that calls some kind
#   of "SET_NAMED_VARIABLE" subroutine, after copying the address of the value to assign
#   (along with the name to assign to) into the argument addresses in the stack
#   for the SET_NAMED_VARIABLE subroutine call
#
#   - The right hand side is evaluated first, as a separate statement (ie: literal or
#     whatever), and then the resulting variable address is used as the 
#
#   - this results in a new entry on the current function stack with the name in question
#     and value in question, ie: we're writing a new value for the variable X onto the end
#     of the current frame stack (*not* subroutine stack)
#
#   - the result (return value) of an assignment is the value that was assigned, ie: the
#     address of the new variable entry in the current stack 
#
# - A function call:
#
#   - evaluate arg expressions as a normal list of statements
#
#   - allocate storage for the function's stack + arguments
#
#   - copy arguments (ie: numeric indexed like an array) into 'argument' space of new stack
#
#   - copy constants into appropriate area of new stack
#
#   - lookup function's top-of-var-stack, ie: at time it was defined (closures), and "restore"
#     it so that's now our top-of-stack
#
#   - bind formal parameter names now that we're inside our new stack
#
#   - create new stack frame (return value, return address, previous frame address)
#     - update frame pointer register
#
#   - execute function body (jump to it) via submode, with the new stack address we've created


# PROGRAM/STACK FORMAT/EXPLANATION...
#
# Overall "memory" layout:
#
#     +------------------------------------------------+
# -R: |  Register addresses                            |
#     |                                                |
#     |    USRxxx - userland registers                 |     * These are thread-specific, and created on demand!
#     |                                                |
# -9: |    SEMGET - integer - semaphores?              |
#     |                                                |
# -8: |    NEWTHR - integer - subtract beginning addr  |     * All registers use negative addresses!
#     |                       of new thread from this  |       - the register space grows *downwards*
#     |                       to create a new thread   |       - the VM does not offset register addresses!
#     |                                                |
# -7: |    MAVAIL - address - first block of usable    |
#     |                       addresses, ie: they have |
#     |                       been freed, can be       |
#     |                       re-allocated             |
#     |                                                |
# -6: |    MALLOC - address - highest used address     |
#     |                                                |
# -5: |    THREAD - integer - current thread id        |  ---+
# -4: |    INSTRP - address - instruction pointer      |     |
# -3: |    SBMODE - address - current subroutine stack |     +--- These are thread-specific!
# -2: |    FRAMEP - address - current frame            |     |
# -1: |    ENDMEM - address - top var in current ns    |  ---+
#     |                                                |
#     +------------------------------------------------+
#  0: |  Program code:                                 |
#     |    os code                                     |
#     |  ..                                            |
#     +------------------------------------------------+
#     |  Program literals                              |
#     |                                                |
#     |   This is a block of storage used for          |     * These are all *top*-level literals - functions
#     |   anonymous values referenced by program code. |       will have their own constant storage space on
#     |                                                |       their stack (part of their body, technically)
#     |   The variable addresses stored here are       |
#     |   filled in to the frame stack of a function   |
#     |   call by the assembly generated for that      |
#     |   function call.                               |
#     |                                                |
#     +------------------------------------------------+
#     |                                                |
# 060 |  Variable storage (Stack begins here)          |
#     |                                                |
#     |  Also subroutine stack storage                 |
#     |                                                |
# 150 |  Also code-allocated 'raw' memory              |
#     |                                                |
#     +------------------------------------------------+

# Code for parsing symbolic register names:
```

In [6]:
from copy import deepcopy


def make_new_register(name, registers, idx):
    # We have to define these as accepting the 'old registers' as arg, even though
    # we already have access to ourself via the 'registers closure' here, but
    # the function prototype for the register getter and setter assumes that
    # the functions will accept the current registers as an arg, ie: for
    # "special" registers...
    
    def getter(old_registers):
        return registers[idx][3]
    
    def setter(x, old_registers):
        registers[idx][3] = x
    
        return registers[idx][3]
    
    return [name, getter, setter, 0]


# This all looks messy, but there's a good reason for it... ie: shared
# vs. thread-specific registers and their accessors

GLOBAL_REGISTERS = [
    ['ENDMEM',],   # These are here as dummy placeholders, so that we can use *just* the 
    ['FRAMEP',],   # global registers when initializing our kernel with the first thread
    ['SBMODE',],
    ['INSTRP',],
    ['THREAD',],
        
    ['MALLOC',],
    ['MAVAIL',],
    ['NEWTHR',],
    ['SEMGET',],
]
GLOBAL_REGISTERS[0] = make_new_register('ENDMEM', GLOBAL_REGISTERS, 0)
GLOBAL_REGISTERS[1] = make_new_register('FRAMEP', GLOBAL_REGISTERS, 1)
GLOBAL_REGISTERS[2] = make_new_register('SBMODE', GLOBAL_REGISTERS, 2)
GLOBAL_REGISTERS[3] = make_new_register('INSTRP', GLOBAL_REGISTERS, 3)
GLOBAL_REGISTERS[4] = make_new_register('THREAD', GLOBAL_REGISTERS, 4)

GLOBAL_REGISTERS[5] = make_new_register('MALLOC', GLOBAL_REGISTERS, 5)
GLOBAL_REGISTERS[6] = make_new_register('MAVAIL', GLOBAL_REGISTERS, 6)


threads = []


# Need to set this up before REGISTERS, since it references the 'special registers' among others...
def new_register_set(start_instrp, old_registers):
    new_registers = build_new_register_set()
    
    set_register_by_name('INSTRP', start_instrp, new_registers)
        
    # set THREAD (based on new thread id) and ENDMEM (copy from 'parent' thread)
    set_register_by_name('THREAD', len(threads), new_registers)
    set_register_by_name('ENDMEM', get_register_by_name('ENDMEM', old_registers), new_registers)
    
    return new_registers

    
def new_thread(start_instrp, old_registers, program):
    new_registers = new_register_set(start_instrp, old_registers)
        
    # Create the new thread
    newthr = Thread(target=run_program, args=(program, new_registers))
        
    threads.append(newthr)
    threads[-1].start()
        
        
def new_thread_get(old_registers):
    return 0
    
    
def semaphore_get(old_registers):
    return 0
    
    
def semaphore_set(x, old_registers):
    return 0
    
    
# Initialize "special" register functions
GLOBAL_REGISTERS[7] = ['NEWTHR', new_thread_get, new_thread]
GLOBAL_REGISTERS[8] = ['SEMGET', semaphore_get, semaphore_set]

REGISTERS = []
REGISTERS += [
    ['ENDMEM',],
    ['FRAMEP',],
    ['SBMODE',],
    ['INSTRP',],
    ['THREAD',],
    
    ['MALLOC', GLOBAL_REGISTERS[5][1], GLOBAL_REGISTERS[5][2]],
    ['MAVAIL', GLOBAL_REGISTERS[6][1], GLOBAL_REGISTERS[6][2]],
    ['NEWTHR', GLOBAL_REGISTERS[7][1], GLOBAL_REGISTERS[7][2]],
    ['SEMGET', GLOBAL_REGISTERS[8][1], GLOBAL_REGISTERS[8][2]],
]

start_reg_len = len(REGISTERS)
    

def build_new_register_set():
    registers = deepcopy(REGISTERS)
    
    registers[0] = make_new_register('ENDMEM', registers, 0)
    registers[1] = make_new_register('FRAMEP', registers, 1)
    registers[2] = make_new_register('SBMODE', registers, 2)
    registers[3] = make_new_register('INSTRP', registers, 3)
    registers[4] = make_new_register('THREAD', registers, 4)
    
    return registers

    
def fill_usr_registers(num, registers):
    rlen = len(registers)
    
    while rlen < abs(num):
        next_usr_num = (rlen - start_reg_len) + 1
        
        registers.append(make_new_register('USR%d' % next_usr_num, registers, rlen))
        
        rlen += 1


def get_register_by_idx(idx, registers):
    fill_usr_registers(idx, registers)
    
    idx = abs(idx) - 1
    
    return registers[idx][1](registers)


def set_register_by_idx(idx, val, registers):
    fill_usr_registers(idx, registers)
    
    idx = abs(idx) - 1
    
    registers[idx][2](val, registers)
    
    return get_register_by_idx(idx, registers)


def get_reg_name(idx, registers):
    positive = (0 - idx)
    
    reg_len = len(registers)
    
    if positive <= reg_len:
        positive -= 1
        
        return registers[positive][0]
    
    else:
        positive -= reg_len
        
        return 'USR%d' % positive
    

class ParseRegException(Exception): pass
def parse_reg_name(name, registers):
    reg_idx = None
    for i in range(len(registers)):
        if name == registers[i][0]:
            reg_idx = i
            break
    
    if reg_idx is None:
        if not name.startswith('USR'):
            raise ParseRegException('Could not parse register name!')
            
        usr_idx = int(name[3:])
        
        return ((0 - len(registers)) - usr_idx)
    
    return (0 - reg_idx) - 1


def get_register_by_name(name, registers):
    idx = parse_reg_name(name, registers)
    
    return get_register_by_idx(idx, registers)


def set_register_by_name(name, value, registers):
    idx = parse_reg_name(name, registers)
    
    return set_register_by_idx(idx, value, registers)


# Test
class TestException(Exception): pass

new_registers = build_new_register_set()

results = []
try:
    results.append(parse_reg_name('USR12', new_registers))
    results.append(parse_reg_name('MALLOC', new_registers))
except Exception as e:
    raise Exception('parse_reg_name: Something is not right - could not parse reg names that should be parseable.')
expected = [-21, -6]

for i in range(len(expected)):
    if expected[i] != results[i]:
        raise Exception('parse_reg_name: Wrong answer: %d (expected) vs %d (actual).' % (expected[i], results[i]))
        
try:
    results.append(parse_reg_name('OOPS', new_registers))
except ParseRegException:
    pass
else:
    raise Exception('Something is not right - parse_reg_name should except on "OOPS" as reg name!')

print('OK!')

OK!


```
#     +-------------------------------------------------+-------------------+-----------------------------+
#     |    A block of subroutine stack                  | substack frame    | A block of userland         |
#     |                                                 | var (length 10)   | -allocated addresses        |
#
#     +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
#  0: |  0 |  0 |  0 |  0 |  0 |  0 |  0 |  0 |  0 |  0 |  A | 10 |  0 |  0 |  0 |  0 |  0 |  0 |  0 |  0 |
#     +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
#                                                     9   10                  14         
#
#       * A points to the address we need to reset SUBMODE to once the subroutine using this stack exits
#
#       * The address 9 is 'returned' inside the code that allocates this first subroutine block - this is
#         the start of the block that was allocated - to reach the substrack frame var, add 4 to this (13)
#
#     +-------------------+--------------------------------------------------------------------------------
#     | user-allocated    |                           This is a function-call stack
#     | block frame var   +--------------------------------------------------------------------------------
#     |                   |    |    |    |       This "first" chunk is all stack used by the function
#                              |    |    |       body and it's subroutines/etc.
#     +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
# 20: |  B |  6 |  0 |  0 |456 |123 |  2 |    |    |    |    |    |    |    |    |    |    |    |    |    |
#     |    |    |    |    |  C |  D |  E |    |    |    |    |    |    |    |    |    |    |    |    |    |
#     +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
#                           24   25   26   27
#
#       * B points nowhere while the storage is in use, points to the last freed frame var once it has been
#         freed itself, and the *next* freed block will then point to it in turn
#
#       * C and D are arguments (single values, usually the address of a variable)
#
#       * E is always the last address in the call stack frame *before* non-argument stack space, and
#         indicates how many arguments were passed. Our function code can access this value at S-1, and
#         from there step down if it needs to enumerate the arguments (the arguments are filled-in in
#         reverse order, so the lowest address in the call stack frame (ie: "first" address in this block)
#         will be the last argument.
#
#       * When this function begins execution (ie: before any subroutines are entered), SBMODE will be 27
#         * All addresses in the function body are relative to this SBMODE value
#
#     --------------------------------------------------------------------------------+-------------------+
#                 This is all still the function call stack                           | function-call     |
#     -----------------------------------------------------------------+----+----+----| frame var         |
#          This portion is still stack for the function code itself                   |                   |
#
#     +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
# 40: |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |  0 | 32 |-80 |  0 |
#     |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |  F |    |  G |    |
#     +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
#                                                                                       56
#
#       * F points to the address of the last frame 'down' in the call stack
#         * Let's say F is zero for now (this is the only frame on the stack)
#
#       * G is our return address (we're cheating here and hi-jacking the usual 'name' place for a variable!
#         * In this case, it's an address elsewhere in the main program (ie: not in a call frame) because
#           this is the first call, but it doesn't really matter what it is (?)
#
#       * When this function returns, the return assembly will update MAVAIL  to point to 59, and 56 will
#         be updated to point to the address of the previously last-freed block (stack/variable/etc)
#
#     +-------------------+-------------------+------------------------+-------------------+--------------+
#     | Named integer var | Anonymous string  | Literal array data     | Another (anon)    | Array Data   |
#     |                   | (array) var       | (var name - a symbol)  | array (3 ints)    |              |
#
#                 --------------------------|
#                 |               ----------|------------------------|
#                 |      |----    |         +------------------------|-----    -------------------------|
#                 |      V   |    |         V                        V    |    |                        V
#     +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
# 60: |  0 | 15 | 67 |  0 | 63 | 72 |  0 |  5 |  o |  l |  l |  e |  h | 67 | 79 |  0 |  3 | 17 | 63 | 82 |
#     +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
#                           64                  68                       73                  77
#
#       * If this were the end of our stack:
#           ENDMEM == 76   the 'last' variable we added to the current namespace (a var address
#
#           FRAMEP == 99   the address of the current 'frame var', ie: containing return-to
#                          address as well as the address of the previous frame var
#
#           SBMODE == 55   before any subroutines *inside* the function in question start executing
#
#           MALLOC == 79   New memory allocations are executed by increasing this, and then writing
#                          a new 'stack var' to store the length and addresses in the avail/etc
#                          linked lists
#
#           MAVAIL == 23   (if this is the last block that was freed)
#                          We first walk this list, looking for a big enough block, when allocating
#                          new memory (if we find one, MALLOC doesn't change, but instead we un-free
#                          the found block and adjust the available linked list)
#
#       * Addresses 24-59 is the call frame, and 56-59 is the 'frame var'


# In order to define a function, we need some info:
#   - the address where the 'body' of the function begins
#     - this is a specially-allocated block that will only be freed
#       explicitly at user request
#     - since the stack is freed on function call frame exit, we need to 'write' the function definition, via
#       this address (will being like all else with a 'variable structure') to a block of userland-allocated
#       storage... ie: the function-defining code allocates storage, then 'writes' the function def into it
#     - this then works like any other user-allocated storage: never freed unless explicitly so
#     - Also, we could theoretically have code that walks the namespace looking for references to a given
#       function, and if none are found, delete that function (ie: if no named variables are 'pointing' to
#       this variable, then it can be assumed there are no more references (???)
#
#   - the length of the body of the function
#
#   - how much 'base' (ie: before userland allocations) stack space is needed, before arguments are included
#
#   - a list of names of any defined formal parameters, in order (ie: corresponding to the order of
#     positional args they are naming)
#
#   - the address of the 'top' of the namespace to which the variable has access, ie: the namespace as it
#     existed at the time of definition of the function - the function's "scope"
#
# A function definition might look like this in memory:
#
#     +------------------------------------------------------+----+----+----+----+----+-------------------+
#     |            Function Body              |  h |  g |  f |  e |  d |  c |  b |  a | Func frame var    |
#
#     +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
# 80: |  x |  x |  x |  x |  x |  x |  x |  x | -1 |  0 |  2 | 50 | 67 |789 |567 |  2 |  0 | 95 | 16 |  2 |
#     +----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
#                                               88   89   90   91   92   93   94   95   96
#
#   - the body length and beginning can be derived from the rest, ie:
#     body length is func name (hijacked for len) - (a + 1) - (f + 1) - 2 = 16 - (2 + 1) - (2 + 1) - 2 = 8
#     body start is val addr + 1 - func name (len) = 95 + 1 - 16 = 80
#
#   - a holds number of arguments
#   - b holds first arg
#   - c holds second arg
#   - d holds ENDMEM to associate with this function (it's scope/namespace)
#   - e holds the number of addresses worth of stack space this function requires when called
#   - f is the number of constants used by the function
#   - g is a constant value
#   - h is a constant value
#
# So in order, to build a function definition in memory:
#
# 1) determine length of body
# 2) add 2
# 3) determine number of args
# 4) add nargs + 1
# 5) allocate current value (ie: 16 in our example, the 4 extra is added automaticaly by the allocating code)
# 6) copy body bytes into beginning (end) of allocation
# 7) determine stack size needed, write that into last body address + 1 (ie: next address)
# 8) determine memory address (ENDMEM) for the stack, write that into next address
# 9) update 'next' value in block frame var to be 0
# 10) update old 'last variable' to point to this block now
# 11) write in formal param names, ie: addresses of symbol strings
# 12) write number of arguments into next address
#
# And in order to 
```

In [7]:

# A simple Klang-SBNZ assembler, with macro expansion

import re


MACROS = {
    'TEST': """
        12, 13, 14,   # MACRO COMMENT
        
    """,
}


test_program = """
A:
    1, -B+1, 3,
    # comment
B:  4, 5, 6, # comment
TEST
C:  7, B, 9,
    7, 8, 9,
"""


ASSEMBLY_CONSTANTS = {
    'ZERO': 0,
    'ONE': 1,
    'TWO': 2,
    'NEGONE': -1,
    'NEGTWO': -2,
}


def inject_constants(idxes):
    retval = [0, 1, None]   # The third element is filled in last, right before we return
                            # This is our jump-past-constants instruction that gets executed
                            # before anything else (ie: )
    
    idx = 3
    
    for k,v in ASSEMBLY_CONSTANTS.items():
        retval.append(v)
        idxes[k] = idx
        idx += 1
        
    while 0 != idx % 3:
        retval.append(0)
        idx += 1
        
    retval[2] = idx
        
    return retval

    
def assemble(program, registers, instrp=0, constants=None):
    labels = {}
    
    assembly = []
    
    if instrp == 0:
        constants = {}
        
        assembly += inject_constants(constants)
    
        instrp = len(assembly)
        
    # Split on newlines
    lines = program.split("\n")
    
    # Loop over each line
    for i in range(len(lines)):
        
        # Strip out comments, strip out whitespace 
        lines[i] = re.sub('#.*$', '', lines[i])
        lines[i] = re.sub('\s+', '', lines[i])
    
        pcs = lines[i].split(',')
        
        # Loop over each address on line
        for j in range(len(pcs)):
            
            if pcs[j] == '':   # Ignore empty places
                continue
                
            label_pcs = pcs[j].split(':')
            
            if 1 < len(label_pcs) and '' != label_pcs[0]:   # successful split for label!
                labels[label_pcs[0]] = instrp
                pcs[j] = label_pcs[1]
                
                if pcs[j] == '':
                    continue
                
            if pcs[j] in MACROS.keys():
                
                assembly += assemble(MACROS[pcs[j]], registers, instrp, constants)
                instrp = len(assembly)
                
            else:
                
                # Check if we're a negative address
                neg = ''
                if pcs[j].startswith('-'):
                    neg = '-'
                    pcs[j] = pcs[j][1:]   # Cut off the minus sign for now
                
                subfinal = None
                
                # First try to parse as a register
                try:
                    subfinal = parse_reg_name(pcs[j], registers)
                    
                except ParseRegException as e:
                    
                    # Check if it's a constant
                    if pcs[j] in constants.keys():
                        subfinal = constants[pcs[j]]
                        
                    else:
                    
                        # Fall back to attempting to parse an integer, with possible label "prepended"
                        # Ie: -A+5, would mean -(A+5) would mean -(100-5) (if A is 100)
                        subfinal = pcs[j]
                    
                if subfinal is None:
                    raise Exception('subfinal should *not* be None!')

                assembly.append('%s%s' % (neg, subfinal))
                instrp += 1
                
    for i in range(len(assembly)):
        neg_factor = 1
        if str(assembly[i]).startswith('-'):
            neg_factor = -1
            assembly[i] = str(assembly[i])[1:]

        for k in labels.keys():
            if str(assembly[i]).startswith(k):
                assembly[i] = '%d%s' % (labels[k], str(assembly[i])[len(k):])
                break
                
        assembly[i] = eval(str(assembly[i]))
        if assembly[i] is not None:
            assembly[i] *= neg_factor
        
    return assembly

        
print(assemble(test_program, GLOBAL_REGISTERS))

[0, 1, 9, 0, 1, 2, -1, -2, 0, 1, -13, 3, 4, 5, 6, 12, 13, 14, 7, 12, 9, 7, 8, 9]


```
# EXECSUB_1() - Execute a subroutine at address X with stack beginning at address Y
#             - The only reason this exists is so that we can avoid writing a function
#               just to copy a chunk of 'code' with an offset, ie: to translate addresses
#               so we can run the same function on a different stack. Instead, we make
#               an atomic (one-instruction) macro that writes to a register which is
#               always zero at write time, which sets the new 'substack' address, and
#               then jumps to the address of the subroutine.
#             - The sneaky bit is the write to a special address - instead of the above
#               described function, we build this functionality into our virtual machine
#               that's executing our code - we just add a 'mode' that means "instead of
#               treating the operands for each instruction literally, assume they are an
#               offset of S, and execute *that* instruction instead". This persists until
#               something (ie: the subroutine) "cancels" sub-mode by zero'ing the submode
#               address (the machine recognizes this address and will not offset it).
#
#             - ** NB: Submode *only* offsets non-register addresses in instruction
#                      position 1 or 2... jumps / values in position 3 are *not* offset!
#               
#       Y, SUBMODE,      X,   # Where SUBMODE is the known address of the submode register


# Mac: ENDSUB_1_0():
#
# SUBMODE, SUBMODE,      0,   # Zero submode
```

In [8]:
MACROS['ALLOC_12'] = """
# Allocate USR1 addresses - re-entrant (can be interrupted - does
#                           not recurse, does not modify local code)
#
#   Args: USR1 holds amount of mem being requested (number of addresses)
#              ie: -2 == 'allocate 2 addresses', holds block start address once complete
#
#   Registers used:  USR1,
#                    USR2 .. USR5,   # misc/etc
#                    USR11, USR10, USR9   # Zero/write instruction
#                    USR8, USR7, USR6   # Jump back instruction
#                    USR12,   # Counter var

A: # Init
   USR2,    USR2,       0,   # Zero USR2
   USR5,    USR5,       4,   # Zero USR5, Four constant
   USR1,    USR5,     A+9,   # USR5 now holds negative # of addresses requested
   USR1,    USR2,    A+12,   # USR2 now holds negative # of addresses requested
    A+2,    USR5,       B,   # USR5 now holds negative # of addresses plus four ('block var' space)

B: # Attempt Malloc loop
   USR3,    USR3,       0,   # Zero USR3, Zero constant
   USR4,    USR4,       1,   # Zero USR4, One constant
 MALLOC,    USR3,     B+9,   # Store negative MALLOC in USR3
   USR5,  MALLOC,       C,   # Allocate space

C: # Verify succes or loop
 MALLOC,    USR4,     C+3,   # Store negative MALLOC in USR4
   USR5,    USR4,     C+6,   # Subtract USR5 from USR4 (subtract allocation)
   USR3,    USR4,       D,   # Subtract USR3 (old neg MALLOC) from USR4,
   ZERO,     ONE,       E,   # JUMP ahead, past loop, if zero (allocation succeeded)

D: # Reset loop, then continue loop w/ B again
   USR3,    USR3,   -USR8,   # Zero USR3, USR8 constant
   USR5,    USR3,     D+6,   # Subtract neg allocation from USR3 (positive value now)       
   USR3,  MALLOC,     D+9,   # De-allocate our chunk
   ZERO,     ONE,       B,   # JUMP to A (loop)

E: # Malloc was success - prep zero'ing USR code...
   USR9,    USR9,   -ZERO,   # Zero USR9, negative address of ZERO (contant)
    D+2,    USR9,     E+6,   # USR9 now holds address of USR8
   USR8,    USR8,    -ONE,   # Zero USR8, negative address of ONE (constant)
   USR7,    USR7,      -G,   # Zero USR7, -G constant
    E+2,    USR8,    E+15,   # Subtract -ZERO from 0 in USR8 to get address of zero constant (register)
    E+8,    USR7,    E+18,   # Subtract -ONE from 0 in USR7 to get address of one constant (register)

# E+18:
   USR6,    USR6,       0,
   E+11,    USR6,    E+24,   # Write G into USR6 for the return from the dynamic writes
  USR12,   USR12,       0,   # Zero USR12 - our loop counter for the block zero'ing loop
   USR5,   USR12,    E+30,   # USR12 now has a positive # of allocated addresses
  USR11,   USR11,       0,
  USR10,   USR10,       0,

# E+36:
   USR3,   USR11,    E+39,   # This set of 4 instructions sets both USR11 and USR10 to the address
   USR5,   USR11,    E+42,   # of the beginning of the allocated block.
   USR3,   USR10,    E+45,   #
   USR5,   USR10,    E+48,   #

F: # Begin zeroing loop
   ZERO,   USR12,   USR11,   # If USR12 is not zero (we're not done), loop (jump to USR11 - perform zeroing write)
   ZERO,     ONE,       H,   # Otherwise, jump to H

G: # USR code for dynamic write (zeroing new block) returns here, this resets the loop and decrements counter.
   # Also decrements addresses being zero'ed, then jumps back to F
    ONE,   USR12,     G+3,   # Subtract 1 from USR12, step
    ONE,   USR11,     G+6,   # Subtract 1 from address at USR11
    ONE,   USR10,     G+9,   # Subtract 1 from address at USR10
   ZERO,     ONE,       F,   # Jump to F (loop)

H: # Done zeroing - set length of block in block-frame var
  USR12,   USR12,       0,   # Zero USR12
   USR1,   USR12,     H+6,   # USR12 now holds negative # of addresses requested (ie: minus block var 4), step
  USR10,   USR10,  -USR12,   # Zero USR10, -USR12 constant
   USR3,   USR10,    H+12,   # USR10 is now original MALLOC addr, step
   USR5,   USR10,    H+15,   # USR10 is not start of new block addr, step
    TWO,   USR10,    H+18,   # USR10 is now the 'value' addr of the blockframe var (len of block), step
# H+18:
  USR11,   USR11,      -I,   # Zero USR11, step
    H+8,   USR11,    H+24,   # USR11 now holds address of USR12
   USR6,    USR6,       0,
   H+20,    USR6,    H+30,   # USR6 now holds address of section I
   ZERO,     ONE,   USR11,   # JUMP to USR11, then back to I from there

I: # Now we need to write our final allocation address back into USR1 (for the "return")
   USR1,    USR1,       0,
   USR3,    USR1,     I+6,
   USR5,    USR1,     I+9,
"""

MACROS['EXIT_0'] = """
A:
    A+3,     A+5,     A+4,
      0,    None,       1,
"""


# Thanks to @brandon_rhodes for this simple colour stuff (tip via twitter.com):
# Custom colour combinations:
yellow_cyan = '\033[33;46m{}\033[0m'.format
blue_magenta = '\033[34;45m{}\033[0m'.format
blue_yellow = '\033[34;43m{}\033[0m'.format
red_yellow = '\033[31;43m{}\033[0m'.format
red_white = '\033[31;47m{}\033[0m'.format
black_red = '\033[30;41m{}\033[0m'.format


TRANSLATE_NON_USR_REGISTERS = True


def colour_instruction(operands, registers, start_of_i=0):
    formatters = [None, None, None]
    
    if operands[0] == operands[1]:   # Zeroing an address, possibly with a constant
        if operands[2] == 0:
            formatters = [yellow_cyan, yellow_cyan, None]
        else:
            formatters = [yellow_cyan, yellow_cyan, blue_magenta]
        
    else:
        if None in operands:
            formatters = [black_red, black_red, black_red]
            
        else:
            oper_formatters = [
                # non-register, USR, non-USR
                [blue_magenta, None, blue_yellow],
                [red_white, None, blue_yellow],
            ]
        
            formatters = []
        
            for i in range(2):
                if operands[i] >= 0:   # A non-register reference, ie: a constant
                    formatters.append(oper_formatters[i][0])
                
                else:
                    reg_name = get_reg_name(operands[i], registers)
                
                    if reg_name.startswith('USR'):   # USR registers
                        formatters.append(oper_formatters[i][1])
                    
                    else:
                        formatters.append(oper_formatters[i][2])
                    
                        if TRANSLATE_NON_USR_REGISTERS:
                            operands[i] = reg_name
                        
            # Process third operand
            if operands[2] < 0:   # USR jump
                formatters.append(blue_yellow)
        
            elif operands[2] != start_of_i + 3:   # Regular jump
                formatters.append(red_white)
            
            else:
                formatters.append(None)
            
    operands = ['%7s' % str(oper) for oper in operands]
    
    return [formatters[i](operands[i]) if formatters[i] is not None else operands[i] for i in range(3)]
        

def assembly_printer(assembly):
    for i in range(0, len(assembly), 3):
        print('%03d: %s, %s, %s' % tuple([i] + colour_instruction(assembly[i:i+3], GLOBAL_REGISTERS, i)))
        

full_test_program = """
A:
     0,       0,    1024,   # 1024 constant (one "kilo" of addresses)
  USR1,    USR1,       0,   # Zero USR1
  USR2,    USR2,       0,   # Zero USR2
   A+2,    USR2,    A+12,   # Subtract 1024 from USR2 (flip sign)
  USR2,    USR1,    A+15,   # Set USR1 to 1024, proceed to ALLOC_12
B:
ALLOC_12
EXIT_0
"""

full_test_program_assembly = assemble(full_test_program, GLOBAL_REGISTERS)

assembly_printer(full_test_program_assembly)

000:       0,       1,       9
003:       0,       1,       2
006:  ENDMEM,  FRAMEP,       0
009:       0,       0,    1024
012:     -10,     -10,       0
015:     -11,     -11,       0
018:      11,     -11,      21
021:     -11,     -10,      24
024:     -11,     -11,       0
027:     -14,     -14,       4
030:     -10,     -14,      33
033:     -10,     -11,      36
036:      26,     -14,      39
039:     -12,     -12,       0
042:     -13,     -13,       1
045:  MALLOC,     -12,      48
048:     -14,  MALLOC,      51
051:  MALLOC,     -13,      54
054:     -14,     -13,      57
057:     -12,     -13,      63
060:       3,       4,      75
063:     -12,     -12,      17
066:     -14,     -12,      69
069:     -12,  MALLOC,      72
072:       3,       4,      39
075:     -18,     -18,      -3
078:      65,     -18,      81
081:     -17,     -17,      -4
084:     -16,     -16,    -129
087:      77,     -17,      90
090:      83,     -16,      93
093:     -15,     -15,       0
096:    

* Now that we have an assembler that works with our definition of registers as negative indices, we need to modify our kernel function(s) to deal with registers appropriately:

In [9]:

from pprint import (
    pprint,
)
import sys
from threading import (
    Thread,
)
from time import (
    sleep,
)


def grow_array_to_idx(arr, idx):
    if len(arr) < (idx + 1):
        arr += [0 for i in range((idx + 1) - len(arr))]
        
    
def read_idx_or_zero(arr, idx, registers):
    # Check for negative index, access registers instead...
    if 0 > idx:
        return get_register_by_idx(idx, registers)
        
    else:
        grow_array_to_idx(arr, idx)
    
        return arr[idx]


def write_idx_or_zero(arr, idx, val, registers):
    # Check for negative index, access registers instead...
    if 0 > idx:
        return set_register_by_idx(idx, val, registers)
    
    else:
        grow_array_to_idx(arr, idx)
    
        arr[idx] = val
    
        return arr[idx]


def run_program_unsafe(program, ip, registers):

    # Assumes:
    #   - 'program' is already an array of integers
    #   - all referenced addresses are populated appropriately (None or integer (0-initialized))
    #   - assumes all non-null operands have values (ie: are integers) and point to
    #     indexes which also contain values (integers)
    #   - etc
    #
    # Also assumes that the instruction pointer will always point to a valid op_0 (and op_1 and
    # op_2 as necessary), and that the instruction pointer is never executing inside register
    # space! THIS LAST POINT IS IMPORTANT! DO NOT BUILD CODE IN REGISTER SPACE if you're using
    # this kernel!!!
    #
    # Aside from error-checking, it doesn't automatically grow the memory space - you need to
    # ensure your program does it explicitly when necessary!
    #
    # Notice there are no function calls here, except for register access - this is a premature
    # optimization for performance (?)... NB: This means that for this particular implementation,
    # register access is more expensive than regular memory access!
    
    while program[ip] is not None:
        op_add_0 = program[ip]
        op_add_1 = program[ip]
        
        op_0 = get_register_by_idx(op_add_0, registers) if 0 > op_add_0 else program[op_add_0]
        op_1 = get_register_by_idx(op_add_1, registers) if 0 > op_add_1 else program[op_add_1]
        
        result = op_1 - op_0
        
        if 0 > op_add_1:
            set_register_by_idx(op_add_1, result, registers)
        else:
            program[op_add_1] = result
            
        if 0 == result:
            ip += 3
        else:
            ip = get_register_by_idx(program[ip + 2], registers) if 0 > program[ip + 2] else program[ip + 2] 
        
    return program[ip - 1]
    
    
# This has been modified to deal with registers appropriately.
def run_program(program, registers):
    """
    Run the program - always run this in it's own thread! (???)
    
    program - an array of integers (our 'source code')
    registers - a newly created register set for this thread
    """
    
    # Validate that program is in fact an array
    
    if type(program) != type([]):
        raise Exception('SBNZ (Klang) programs must be passed as arrays!')
        
    # Step through every address and make sure it's an int (or None)
    program = [int(i) if i is not None else None for i in program]
        
    ip = get_register_by_name('INSTRP', registers)
    
    op_1_address = read_idx_or_zero(program, ip, registers)   # Read the next instruction's first operand
    
    # op_1_address, ie: value at the instruction pointer, is an address
    # pointing to the 2nd subtraction operand
    
    while op_1_address is not None:
        
        # operand 2 is an address of the value to use as first subtraction operand, and
        # operand 3 is the address to jump to if non-zero... so both of these have to be
        # non-null, ie: non-None (or at least operand 2 if the result is going to be zero).
        
        op_2_address = read_idx_or_zero(program, ip + 1, registers)
        op_3_address = read_idx_or_zero(program, ip + 2, registers)
        
        if op_2_address is None:
            raise Exception('SBNZ (Klang) requires operand 2 to be a valid address!')
        
        # the values "pointed" to by op_1_address and op_2_address should be integers (not None), and
        # so we validate this as well (non-null) and then cast to integer
        
        op_1_value = read_idx_or_zero(program, op_1_address, registers)
        op_2_value = read_idx_or_zero(program, op_2_address, registers)
        
        if op_1_value is None or op_2_value is None:
            raise Exception('SBNZ (Klang) requires each instruction to have 3 (at least two) valid integer operands!')
        
        #op_1_value = int(op_1_value)
        #op_2_value = int(op_2_value)
           
        # Perform subtraction (2nd operand - 1st operand)
        
        result = op_2_value - op_1_value
        
        # DEBUG ONLY 
        #print('%03d: %7d, %7d, %7d     /     %7d, %7d, %7d' % (ip, op_1_address, op_2_address, op_3_address, op_1_value, op_2_value, result))
        
        # Store result in the address pointed to by 2nd operand
        
        write_idx_or_zero(program, op_2_address, result, registers)
        
        if result == 0:   # Subtraction result is 0 - do not jump
            ip += 3
            
        else:
            if op_3_address is None:
                raise Exception('SBNZ (Klang) requires a valid address for operand 3, where a jump occurs.')
            
            ip = op_3_address   # Jump to instruction beginning at op_3_address
        
        # Prep for next iteration - read the new op_1_address  
        op_1_address = read_idx_or_zero(program, ip, registers)

    # DEBUG ONLY
    #print('Thread registers at exit...')
    #pprint(registers)

    # The result of the program, once we hit a None for operand 1 of the next instruction, is the
    # value at current-instruction-pointer-minus-one
    
    return read_idx_or_zero(program, ip - 1, registers)


def start_kernel_prep(program):
    # Initialize "end of memory" pointer (MALLOC)
    set_register_by_name('MALLOC', len(program) - 1, GLOBAL_REGISTERS)
    
    # Update function for set NEWTHR to have a reference to the program...
    def newthr_set_f(x,y):
        return new_thread(x, y, program)
    
    GLOBAL_REGISTERS[7][2] = newthr_set_f
    
    
def start_kernel(program):
    # HERE @TODO Make this version of the kernel explicitly NOT support threads! (?)
    
    start_kernel_prep(program)
    
    new_registers = new_register_set(0, GLOBAL_REGISTERS)

    run_program(program, new_registers)
    
    
def start_kernel_threaded(program):
    start_kernel_prep(program)
    
    # Create the thread!, ie: write to it to trigger a new thread creation
    set_register_by_name('NEWTHR', 0, GLOBAL_REGISTERS)
    
    while any([t.is_alive() for t in threads]):
        sleep(0.1)   # Sleep for 100 msec before checking threads again

        
start_kernel(full_test_program_assembly)
#start_kernel(full_test_program_assembly)





Let's stop for a moment and examine what we're trying to accomplish - we want the basis for a "meta" language of sorts with which we can build any reasonable language, both in the context of syntax and semantics, and due to the goal of being as user-extensible as possible, we want this base meta-language to be as simple as possible, for various reasons...

And so we've devolved into an exploration of the very basic requirements for turing-complete computation, and we now have a one-instruction set virtual machine specification, and are starting to build higher-level abstractions on top of that. But we're still not even close to a high-level language that would be comfortable for an end-user to use - we've basically created a very obtuse, archaic machine language that's rather tedious to use.

So how do we get from our bare-bones OISC, to a usable but still bare-bones and highly-extensible, comfortable high-level language?

We need to now examine what the features are of a "comfortable", high-level language. Let's try to extrapolate from our current, very *low* level bare-bones turing requirements:

- we need to be able to write to arbitrary memory locations
- we need to be able to write arbitrary values to those locations
- we need to be able to execute code conditionally - jump to another location in the code to continue execution, which implies potentially 'skipping' some code
- we need to be able to "advance execution" in some well-defined way, ie: "execution proceeds from left to right through the ordered list of instructions, with the exception of explicit jumps"

There are a few other implied requirements, which stem partly from the above four:

- we need a way to refer to a given memory location for execution purposes, ie: jumps
- we need a way to represent values
- we need a way to use human-readable names for memory locations, because humans are bad at remembering lots of numbers

- a program is a *ordered* sequence... and since thus far at least we treat data and code the same (memory locations with values), this boils down to a single requirement: we need to be able to represent lists of things as well as single things (atoms) - thus, we want a syntax for specifying a list of things that is understood to mirror the fashion in which a list is 'modelled' in our assembly - namely, as a consecutive sequence of addresses

- if we're representing lists, we need some indicator of where a list begins and ends, at least syntax-wise for the purposes of parsing

Consider the following code snippet in an imaginary language (scroll past the comments immediately below):

```
#     +--------+--------+--------+--------+--------+--------+--------+--------+
# 18: |     19 |      7 |     21 |      4 |     23 |     18 |        |        |
#     +--------+--------+--------+--------+--------+--------+--------+--------+
#
# - if "addrof 'a'" returns 18, and "addrof 'b'" returns 20, and c is at 22, then:
#
#   - a bare 'a' will get translated into 19
#
#   - "b = a" will get translated into "20 = 18", which means "set address 20 to have the value of address 18"
#
#   - "c = addrof 'a'" will result in address 23 having the value 18 (the address of the 'a' variable itself, not
#     the value of 'a') - see above for the result in address 22 and 23
#
#   - c on it's own would then have the value 18
#
#   - "deref c" would then return the thing that c points to:
#
#     - c's address is 22, which has value 18
#
#     - 18 is thus passed to deref, and deref then returns the value of this new address 18, which is 19
#
#     - 19 points to the value of a, which is what we're after, so the result of deref needs to be treated as
#       the result of a bare variable, namely as the address holding the 'final' value we're after
#
#     - this allows 'deref' on the left hand side of an assignment, since like a regular variable, it just
#       gets translated into something that provides an address for a value before the assignment acts on it

(
  a = 1   # <addr named 'a'> is set to the value of <addr of constant 1>
  b = a   # <addr named 'b'> is set to the value of <addr named 'a'> (b is now 1)

  c = 0
  old_mem = addrof 'c'   # old_mem now holds the *current* address of the value named 'c'
  
  addrof 'c' = malloc 3   # We are calling two subroutines here: addrof and malloc.
                          # - addrof takes a value (a variable name as an integer) and 'returns' the address whose
                          #   name is c, ie: the address whose value is the address of the variable 'c'
                          # - malloc allocates X locations, and returns the (bottom) address of the allocated storage
                          # So, what we're doing is allocating 3 locations, and making the variable 'c' have the
                          # address of the allocated storage instead of it's old address.

  free old_mem   # Since we re-pointed C, there's now an unused address hanging around that held it's initial
                 # value, so we mark it as free'd with a subroutine
  
  d = addrof 'a'   # 'a' (single quotes) tells the parser to convert it into the byte code for ascii 'a', ie: an int,
                   # so that it doesn't instead write code to convert 'a' into a normal variable address before
                   # being passed to addrof
  
  c = 2   # c is being used as an array of arguments... 2 is the number of subsequent elements
  c+1 = b
  c+2 = 1
  
  if (lt(b, 5), jmp(3))
  
  print b
  
  
  
  # Final, friendly version:


  # A macro definition...
  
  MACRO LABEL
    a bunch of text
    more text
    etc
  END
  
  # or
  
  MACRO LABEL_TWO some more text END

  # Declare variables so that the ordering (layout) in memory is known (ie: for writing dynamic instructions).
  # (Parser understands assignment to null, and multiple assignment, so that this just results in pre-mapping
  #  the variable names to sequential USR register addresses.)
  # The variables themselves (ie: symbols and their values (address of variable value)) get written into
  # non-executing... what??? (This sentence was unfinished... non-executing memory I think was the intent... via the parser/compiler...)
  
  a , b , c ,
  d , e , f ,
  g , h
  
  # Direct assignments using a constant value expand to two instructions (one if the constant is zero):
  # - Zero the address being assigned to,
  # - subtract address of corresponding negative constant (-1 for the following assignment))...
  
  a = 1   # a is 1
  
  # Assignments using a variable expand to four instructions:
  # - Zero the 'COPY' register
  # - Zero the bindee (destination variable)
  # - subtract value of variable address from copy register
  # - subtract value of copy register from bindee
  
  b = a   # b is 1
  
  # address assignments require 2 instructions - they are translated into constant (register address)
  # assignments:
  # - Zero the address being assigned to
  # - subtract the negative address constant for 'b', ie: -USR2 or whatever
  c = &b   # c points to address of b
  
  # dereferencing a pointer requires 29 (34) instructions (assembly - the 34 refers to *executed*
  # instructions, ie: produced assembly plus instructions executed in USR space):
  #
  # Instruction layout (for "dynamic" instructions built & executed in USR space):
  #
  #    A:  COPY,  COPY,     0,   # Requires fifteen addresses total (in USR space), beyond the assembly itself  
  #  A+3:   DST,   DST,     0,   # that uses them...
  #  A+6:   SRC,  COPY,   A+9,
  #  A+9:  COPY,   DST,  A+15,
  # A+12:  ZERO,   ONE,  A+15,
  # A+15: <END>
  #
  # So we then need 29 instructions total for a dereference:
  #
  # - four more instructions to zero each instance of DST or SRC
  #
  # - one instruction to zero a temp var (one of the noop zero jumps in the 12 above USR addresses)
  # - one instruction to subtract pointer val (pointee address) from tmp var
  # - one instruction to subtract negative pointer val in tmp var from the one instance of SRC
  #
  # - three instructions to subtract negative DST val (negative USR constant) from each instance of DST
  # - two instructions to zero return addresses (A+11, A+14)
  # - two instructions to write return addresses
  # - eight instructions to zero and then write each instance of COPY
  # - four instructions to zero and then set the constants in the last instruction for the unconditional
  #   jump (A+12 and A+13)
  # - two instructions to zero and then write the A+9 address into A+8
  # - lastly... one instruction to jump unconditionally to the beginning of our dynamic instructions
  
  d = *c   # d is the value of the thing c points to, ie: b, ie: 1
  
  # assigning to a dereference requires 29 instructions (compiled, plus 5 in USR space):
  #
  # Instruction layout is the same as that for a dereference assignment in USR space, ie: after the
  # pointer operations, it reduces to a regular 'assign named variable' set of four instructions plus
  # a fifth for the return.
  #
  #    A:  COPY,  COPY,     0,   # Requires fifteen addresses total (in USR space), beyond the assembly itself  
  #  A+3:   DST,   DST,     0,   # that uses them...
  #  A+6:   SRC,  COPY,   A+9,
  #  A+9:  COPY,   DST,  A+15,
  # A+12:  ZERO,   ONE,  A+15,
  # A+15: <END>
  #
  # So we then need 29 (34) instructions total for a dereferenced assignment (same as assigning FROM a deref):
  #
  # - four more instructions to zero each instance of DST or SRC
  #
  # - one instruction to zero a temp var
  # - one instruction to subtract the dereferencee address from a temp var
  # - three instructions to subtract the negative dereferencee address from each instance of DST
  #
  # - one instructions to subtract negative SRC val (negative USR constant) from the instance of SRC
  # - two instructions to zero return addresses (A+11, A+14)
  # - two instructions to write return addresses
  # - eight instructions to zero and then write each instance of COPY
  # - four instructions to zero and then set the constants in the last instruction for the unconditional
  #   jump (A+12 and A+13)
  # - two instructions to zero and then write the A+9 address into A+8
  # - lastly... one instruction to jump unconditionally to the beginning of our dynamic instructions
  
  *c = 2   # b (the thing c points to) is now 2 (c still points to b, ie: it's value is the address of b)

  # This should not be allowed (?) ! IGNORE FOR NOW
  # &e = 3   # e now has the address 3 - e's value is whatever value is at address (location) 3
  
  g = 0   # We've already covered this
  
  # Assignments with addition/subtraction require one instruction (using constants)

  g -= 2   # g is -2 - in our OISC, += and -= are just alternate forms of assignment since it's all
           #           working the same way under the hood
  
  # Assignments with addition/subtraction require either one instruction (subtraction) or three
  # (addition, uses COPY reg, which needs zero'ing, then writing, then subtracting from dest)... if
  # we're assigning from variable values and not constants (see above for constants)
  
  g += a   # g is -1
  
  _LOOP1_:   # Address label, for jumps
  
  g -= -1   # g is 0 (first iteration)
  h = g
  
  # An assignment (any assignment) that has an optional jump included still uses the same number
  # of instructions as without the jump... the only difference is an "explicit" jump destination
  # is used, instead of the parser automatically calculating it (ie: as the next sequential instruction)
  
  h -= 5 _LOOP1_   # This is a conditional jump if the assigned value is nonzero
  
  # g is now 5
)
```

Array access/length/etc can be built on top of these primitives using pointer arithmetic/etc, as can all other operations like implementing higher-level data structures of other kinds (ie: functions), defining functions,
calling functions, control-flow constructs, etc.

## So let's build our language now!

We have a *relatively* human-friendly syntax we can now use, which is isomorphic to the abilities/operations we can perform via our OISC Klang assembly language, which means we just need a parser for our "intermediate" language, ie: the bare-bones, "high-level" language from which all else will be built inside. Ie: ultimately, the goal is to have the language bootstrap it's own parser, and then support new userspace constructs/syntax/etc via macros (parser "plugin" functions).

Assuming this slightly higher-level language (bare-bones Klang) is functionally isomorphic to Klang assembly (it should be), we can assume it is also Turing-complete just like Klang assembly, which means we still have all the pieces we need, but now in a human-friendly format. And since this is Turing-complete... we can use it to build any other type of computation... or more specifically, any other type of **language**, as long as the language can be modelled by a Turing-complete machine.

We have our meta-language! But... it's still fairly pedantic and low-level. What we need now is to build newer, higher-level constructs with the low-level operations we already have available to us, as well as implement associated requirements/etc... build all the different "blocks", like function definitions, subroutines, dictionary structures, whatever, that we can then mix and match (and possibly add to other, new constructs) in order to produce a given language (with a given syntax, based on parse macros/etc... more on that later).

So our current (next) requirements:

- write a parser (modify what we have so far) so that it's capable of parsing and *assembling* the above sample syntax

- implement ability to write macros in bare-bones Klang, as well as Klang assembly
  - @TODO HERE - both is unnecessary - support for one (bare-bones) implies support for the other (???)
  - *our existing parser for Klang assembly already supports macro expansion*

- write a formal language definition for bare-bones Klang
  - @TODO HERE - we *sort* of have some of this already - see comments below which precede the parser/compiler code!

- start implementing higher-level constructs in bare-bones Klang via macros, that can then be combined/extended to produce more capable, higher-level "language sets"

  - function definitions and calls:
    - need to determine (define) the semantics of variable scope/etc
    - but wait!!! This is just a language design decision - we can build different, higher level languages on top of bare-bones Klang for different semantic requirements. Maybe one language supports closures, maybe one doesn't, etc.???
  - etc.

- once we're bootstrapped up to a 'full' high-level language **at least to the point where it supports userland macros**, re-implement parser and all base macros (bare-bones or assembly) as userland functions (macros) ?????

The idea is this: The core Klang language (kernel/virtual machine/etc) is an implementation of *only* the bare-minimum requirements for computation, and everything else is built up from that, ie: our high-level language(S). The reason for the extreme simplicity is that we're aiming for a language that can quickly and easily be used to build other higher-level languages - from experience, most programmers will be able to tell you that the simpler something is (ie: an algorithm/etc), the easier it is to 1) troubleshoot/debug/fix, 2) compose with other things. **this needs more clarification/explanation... still pretty vague/assumption-ridden**



In [10]:
# Bare-bones Klang assembler

# We can modify a lot of our existing Klang assembler,
# but we'll also need to change a lot, so let's start
# fresh and just take what we can as we proceed

# NB: We're still just building an *assembler*, **not** a parser, so things
#     like high-level macro expansion and re-parsing after macro definitions,
#     etc, do not come into play here - ultimately, any high-level language
#     we build upon bare-bones Klang will have a parser that produces some
#     kind of AST, which is where all user-level parsing/macro stuff happens,
#     and then some form of high-level assembler will take the AST and
#     "statically" convert that into bare-bones Klang, which is then passed
#     to this parser.
#
#     Main practical thing that comes from that is that all our assembly-level
#     macros (and thus bare-bones macros as well - things which produce assembly)
#     are defined statically ahead of time as part of the core Klang assembly/
#     bare-bones kernel. Remember... bare-bones Klang still reduces to assembly
#     **just like** Klang assembly, it's just a slightly friendlier syntax, but
#     at the end of the day we're still writing assembly, it just doesn't look
#     like it because the assembler does a lot of translation for us to make
#     what we're writing (exhaustive assembly) easier to read (bare-bones Klang)

# So, our language is defined thusly:
#
# - 'statements' are separated by newlines, except in the case of variable lists
#   (ie: comma preceding a newline, which is only valid as part of a variable
#    list, serves to 'join' the next line to this one for parsing purposes)
#
# - comments begin with '#' and continue until end of line
#
# - a non-empty statement (ie: a line in the source with more than just whitespace
#   and/or comments) can be EITHER an assignment (operation), a label, or a variable
#   list
#
# - labels (definitions) must be sequences of A-Za-z0-9_, and must appear on a line by
#   themselves, and MUST include a trailing colon (':'), which is NOT part of
#   the label but rather an indicator
#
#   - EXCEPT for the first character, which can't be a digit (numeric)!
#
# - an assignment is a statement that contains either '=', '-=', or '+=' (all of
#   which must be surrounded by whitespace), and must contain non-empty right and
#   left hand sides (after leading and trailing whitespace, and comments, are stripped
#   from the line)
#
#   - an assignment has an optional tail, which is a label for the possible jump,
#     otherwise the parser/compiler will automatically use the next sequential
#     instruction address (current instruction + 3) to fill in the possible jump
#     operand automatically
#
# - left and right-hand sides of an assignment must be variable references (left) or
#   constant integers (right only)
#
# - variable names follow the same rules as label names, minus the colon
#
# - a variable reference is either:
#
#   - a bare variable (same rules as label names minus the colon)
#
#   - a variable deref (valid on either left or right hand side), which is a variable
#     name preceded by a '*'
#
#   - a variable address (valid only on the right hand side), which is a variable
#     name preceded by a '&'

from functools import reduce
from pprint import pprint as pp
import re


PREAMBLE = [0, 1, None]   # 3rd address (initial jump to start of code) filled in at the end
VAR, LABEL, ASSIGNMENT = 0, 1, 2
SUB, ADD, DIRECT = 0, 1, 2   # Assignment type
VAR_OPER, CON_OPER = 0, 1   # Assignee (operand) type - either a variable reference or a constant
REGISTER, CONSTANT, VARIABLE = 0, 1, 2

VAR_AND_LABEL_REGEX = '[a-zA-Z_][a-zA-Z0-9_]*'
LEFT_VAR_REFERENCE = '\*?%s' % VAR_AND_LABEL_REGEX
RIGHT_VAR_REFERENCE = '(\*|\&)?%s' % VAR_AND_LABEL_REGEX


def declare_variables(new_vars, compiled, var_map, instrp):
    pass   # HERE


def lines_and_tokens(program):
    # strip leading/trailing whitespace, split on new lines
    program_lines = re.split('\r?\n', program.strip())
    
    # strip comments and then leading and trailing whitespace for each line
    program_lines = list(map(lambda x: re.sub('\s*#.*$', '', x).strip(), program_lines))
    
    # filter out empty lines
    program_lines = list(filter(lambda x: x != '', program_lines))
    
    # Split on whitespace for an array of arrays (lines and tokens for each line)
    program_lines_with_tokens = list(map(lambda x: re.split('\s+', x.strip()), program_lines))
    
    # Now we need to find any lines that end with a ',', and join them to the following line...
    
    acc = []   # accumulator - essentially the following is a reduce, but easier to read this way...
    
    for i in range(len(program_lines_with_tokens)):   # for each line index...
        
        if len(acc) and acc[-1][-1] == ',':   # if there is an existing line, and it ends with a comma
            acc[-1] += program_lines_with_tokens[i]
            
        else:
            acc.append(program_lines_with_tokens[i])
            
    program_lines_with_tokens = acc   # Unnecessary - just to be clear, ie: consisten var names both
                                      # inside and outside functions for a given data structure/value!
    
    return program_lines_with_tokens
            
    
# Macro definitions are parsed out before anything else, and cannot be nested/recursive!
def build_and_strip_macros(program):
    macros = {}
    stripped_program = ''
    
    macro_pcs = re.split('MACRO\s+', program)

    if 2 > len(macro_pcs):   # No macros found...
        return (macros, stripped_program)
    
    # We found a macro, ie: there was a split...
    stripped_program = '%s%s' % (stripped_program, macro_pcs[0])
        
    for i in range(1, len(macro_pcs), 1):   # We're splitting on the MACRO start, so odd indexes
                                            # are the macro bodies with trailing ' END'
        
        stripped_regex = re.compile('.*END', re.DOTALL)
        stripped_program += re.sub(stripped_regex, '', macro_pcs[i])
                
        macro_regex = re.compile('END.*', re.DOTALL)
        macro_all = re.sub(macro_regex, '', macro_pcs[i]).strip()

        space_idx = macro_all.find(' ')
        newline_idx = macro_all.find("\n")
        return_idx = macro_all.find("\r")
            
        idxes = [space_idx, newline_idx, return_idx]
        pos_idxes = list(filter(lambda x: x >= 0, idxes))
            
        max_idx = max(*idxes)
            
        if -1 < max_idx:   # We found a space    
            key_token = macro_all[:min(pos_idxes)]
            body = macro_all[min(pos_idxes):]
            
            # Simple check for obviously recursive macros - doesn't help co-recursion though :(
            if 0 <= body.find(key_token):
                raise Exception('Could not expand - macro is recursive! (%s)' % key_token)
            
            macros[key_token] = body
                
        else:   # No space... this is an empty macro
            macros[macro_all] = ''
            
    return (macros, stripped_program)
    
    
def assemble(program, registers, constants_and_compiled=None):
    # Init these, so we can recurse...
    if constants_and_compiled is None:
        constants = []
        compiled = []
    else:
        constants, compiled = constants_and_compiled
    
    
    program_lines_with_tokens = lines_and_tokens(program)
    
    
    # Now we have our list of statements tokenized, we can start to compile!
    
    var_map = {}
    label_map = {}
    
    constants = []
    compiled = []
    
    
    
def parse(stripped_program, registers):
    program_lines_with_tokens = lines_and_tokens(stripped_program)
    
    ast_list = []
    
    for i in range(len(program_lines_with_tokens)):
        line = program_lines_with_tokens[i]
        
        line_len = len(line)
        
        if line_len == 1:   # either a single variable (for declaration) or a label...
            
            if line[0].endswith(':'):   # a label
                label = line[0][:-1]
                
                if not re.match(VAR_AND_LABEL_REGEX, label):
                    raise Exception('Invalid label: %s' % label)
                    
                ast_list.append([LABEL, line[0][:-1]])
                
            else:   # a single variable (for declaration)
                ast_list.append([VAR, line[0]])
            
        else:   # either a variable list (for declaration) or an assignment...
            
            if ',' in line:   # a variable list
                line = list(filter(lambda x: x != ',', line))
                
                for var in line:
                    
                    if not re.match(VAR_AND_LABEL_REGEX, var):
                        raise Exception('Invalid var name: %s' % var)
                    
                    ast_list.append([VAR, var])
                    
            else:   # an assignment
                
                if line[1] == '-=':
                    oper = SUB
                elif line[1] == '+=':
                    oper = ADD
                else:
                    oper = DIRECT
                    
                # Left is always a variable reference of some kind...
                
                if not re.match(LEFT_VAR_REFERENCE, line[0]):
                    raise Exception('Invalid left var reference: %s' % line[0])
                   
                l_deref = False
                if line[0].startswith('*'):
                    l_deref = True
                    line[0] = line[0][1:]
                    
                try:
                    subfinal_left = parse_reg_name(line[0], registers)
                    subfinal_left_type = REGISTER
                
                except ParseRegException as e:
                    subfinal_left = line[0]
                    subfinal_left_type = VARIABLE
                    
                # Right is a reference or a constant
                
                r_deref = False
                r_addrof = False
                
                if line[2].startswith('*'):
                    r_deref = True
                    line[2] = line[2][1:]
                    
                elif line[2].startswith('&'):
                    r_addrof = True
                    line[2] = line[2][1:]
                
                # Strip off possible leading minus sign, temporarily...
                neg = False
                if line[2].startswith('-'):
                    neg = True
                    line[2] = line[2][1:]
                
                try:
                    subfinal_right = parse_reg_name(line[2], registers)
                    subfinal_right_type = REGISTER
                    
                except ParseRegException as e:
                    try:
                        subfinal_right = int(line[2])
                        subfinal_right_type = CONSTANT
                        
                    except Exception as e:
                        subfinal_right = line[2]
                        subfinal_right_type = VARIABLE
                        
                ast_list.append([ASSIGNMENT,
                                 oper,
                                 subfinal_left,
                                 subfinal_left_type,
                                 l_deref,
                                 subfinal_right,
                                 subfinal_right_type,
                                 r_deref,
                                 r_addrof,
                                 neg])
                
                # JUMP is a reference or a constant
                
                if len(line) > 3:
                    j_deref = False
                    j_addrof = False
                
                    if line[3].startswith('*'):
                        j_deref = True
                        line[3] = line[3][1:]
                    
                    elif line[3].startswith('&'):
                        j_addrof = True
                        line[3] = line[3][1:]
                
                    # Strip off possible leading minus sign, temporarily...
                    neg = False
                    if line[3].startswith('-'):
                        neg = True
                        line[3] = line[3][1:]
                
                    try:
                        subfinal_jump = parse_reg_name(line[3], registers)
                        subfinal_jump_type = REGISTER
                    
                    except ParseRegException as e:
                        try:
                            subfinal_jump = int(line[3])
                            subfinal_jump_type = CONSTANT
                        
                        except Exception as e:
                            subfinal_jump = line[3]
                            subfinal_jump_type = VARIABLE
                    
                    ast_list[-1] += [subfinal_jump,
                                     subfinal_jump_type,
                                     j_deref,
                                     j_addrof,
                                     neg]
                    
                # Make sure we're not using &/* together with neg!
                if (ast_list[-1][7] or ast_list[-1][8]) and ast_list[-1][9]:
                    #pp(ast_list)   # [2, 2, 'a', 2, False, 1, 1, False, False, False]]
                    raise Exception('Using a negative operand to addrof or deref is invalid: %s' % ' '.join(line))
                    
                # Same thing for JMP, if there is one!
                elif len(ast_list[-1]) > 10 and (ast_list[-1][12] or ast_list[-1][13]) and ast_list[-1][14]:
                    raise Exception('Using a negative jump operand with addrof or deref is invalid: %s' % ' '.join(line))
                    
    return ast_list


def ast_list_to_code(ast_list, registers):
    code = ''
    
    for ast in ast_list:
        
        if ast[0] == VAR:
            code = "%s%s\n" % (code, ast[1])
            
        elif ast[0] == LABEL:
            code = "%s%s:\n" % (code, ast[1])
            
        else:   # ASSIGNMENT

            if ast[4]:   # left deref
                code = "%s*" % code
                
            code = "%s%s" % (code, ast[2])   # subfinal_left
            
            if ast[1] == SUB:
                code = "%s -= " % code
            elif ast[1] == ADD:
                code = "%s += " % code
            else:
                code = "%s = " % code
                
            if ast[7]:   # right deref
                code = "%s*" % code
            elif ast[8]:   # right addrof
                code = "%s&" % code
                
            if ast[9]:   # neg
                code = "%s-" % code
            
            if REGISTER == ast[6]:
                code = "%s%s" % (code, get_reg_name(ast[5], registers))
            else:
                code = "%s%s" % (code, ast[5])
                
            if 10 < len(ast):
                if ast[12]:   # jump deref
                    code = "%s *" % code
                elif ast[13]:   # jump addrof
                    code = "%s &" % code
                else:
                    code = "%s " % code
                    
                if ast[14]:   # neg
                    code = "%s-" % code
                    
                if REGISTER == ast[11]:
                    code = "%s%s" % (code, get_reg_name(ast[10], registers))
                else:
                    code = "%s%s" % (code, ast[10])
                
            code = "%s\n" % code
            
    return code
            

def rewrite_vars(stripped_program, rewrite_num, registers):
    ast_list = parse(stripped_program, registers)   # Parse the (macro def-stripped) source into an ast_list we can walk
            
    for i in range(len(ast_list)):   # For each line in the ast_list structure
        
        # old_idxes is a list of the indexes in the current line which contain variable
        # names that need rewriting
        old_idxes = []
    
        if ast_list[i][0] in (VAR, LABEL):   # This line is either a variable declaration or a label
            old_idxes.append(1)
                    
        else:   # This line is an assignment
            
            if ast_list[i][3] == VARIABLE:   # Don't rewrite registers!
                old_idxes.append(2)
                
            if ast_list[i][6] == VARIABLE:
                old_idxes.append(5)
                
            if ast_list[i][11] == VARIABLE:
                old_idxes.append(10)
        
        for j in range(len(old_idxes)):   # each index on this line that needs re-writing...
            
            old_idx = old_idxes[j]
            
            old_var = ast_list[i][old_idx]   # value that needs rewriting
            
            if old_var not in rewritten.keys():   # if we don't already have a new name for this variable...
                new_var = "%s_%d" % (old_var, rewrite_num)
                rewritten[old_var] = new_var
                rewrite_num += 1
            
            # write in the new variable name based on the mapping we already had or just created...
            
            ast_list[i][old_idx] = rewritten[old_var]
           
    return (ast_list_to_code(ast_list, registers), rewrite_num)   # Convert the ast_list back into source code and return it
    
    
def expand_one_macro(stripped_program, macros, registers, rewrite_num, do_rewrite=False):
    mkeys = list(macros.keys())
    keyslen = len(mkeys)
    
    rewritten = {}
    
    for i in range(keyslen):
        macro_key = mkeys[i]
                   
        idx = stripped_program.find(macro_key)
                   
        if -1 < idx:   # We found a macro that needs replacing!
            
            head = stripped_program[:idx]
            tail = stripped_program[(idx + len(macro_key)):]
            
            macro_body = macros[macro_key]   # We need to re-write variable names inside the macro body!
            
            if do_rewrite:
                (macro_body, rewrite_num) = rewrite_vars(macro_body, rewrite_num, registers)
            
            stripped_program = head + macro_body + tail
            
            return (stripped_program, True, rewrite_num)   # The True indicates we need to re-execute (there still
                                                           # might be more macros, because we found one)
            
    return (stripped_program, False, rewrite_num)

    
def compile(program, registers):
    
    # First, we need to pull out and build any macros...
    
    (macros, stripped_program) = build_and_strip_macros(program)
    
    # Before we expand macros, we need to parse the source so far so we can maintain a
    # list of variable names while we expand, ie: so we can re-write variable names
    # inside the macros...
    
    ast_list = parse(stripped_program, registers)
    
    # Next, we need to expand all macros...
    
    rewrite_num = 0
    
    (stripped_program, more, rewrite_num) = expand_one_macro(stripped_program, macros, registers, rewrite_num)
    
    while more:
        (stripped_program, more, rewrite_num) = expand_one_macro(stripped_program, macros, registers, rewrite_num)
        
    # HERE - We now implement a two-phase compile of the AST to Klang machine code
    #
    #        - first, we do one compilation pass to assemble the final code, except we do not
    #          rewrite any variable references or constants yet... instead, each distinct
    #          variable reference ('a' and '*a' and '&a' are all distinct!) is tracked in a hash
    #          according to some coding (ie: CONST:-1, CONST:0, VAR:a, NEGVAR:b, ADDROF:c, DEREF:d, etc),
    #          and we assemble using the codes instead.
    #
    #        - once we have a list of all the variables and constants in use for a given "program"
    #          (ie: how many variables are in use, and how many constants), we can figure out how
    #          many addresses are needed to store ((num variables + num constants) * 2, since we
    #          store both negative and positive versions of each distinct absolute value)
    #
    #        - we *always* start the main kernel code with "0, 1, <first instruction after constants>"
    #
    #        - we also try to pack constants into unused jump addresses, ie: the jump addresses of
    #          zeroing instructions (there op1 and op2 are the same address)
    #
    #        - so we track where each 'code' has been stored, and then once we have the full 'map',
    #          we can proceed with the second phase, which is going through the compiled code and
    #          replacing these "constant codes" with actual addresses.
    
    # A piece of code can always be compiled to run at a given offset, so that we can "live compile"
    # code into a running kernel and then execute (ie: "import libraries", REPL, etc)
    #
    # - when a piece of code is compiled for the purpose of linking into an existing program like this
    #   we optionally compile it *with* existing variable/constant mapping (if we want it to run in
    #   the "scope" of the existing code, ie: a variable named "abc" in existing code would be
    #   referenced by an "abc" in the newly compiled code, instead of a separate/new "abc" variable).
    #   If we want all non-register address access to refer to newly allocated addresses, then we
    #   compile *without* the existing variable mappings - a 'dynamic' compilation that can be linked
    #   into any program.
    #
    # - the former case, where we link the code into the existing code (kernel), is like a C 'static'
    #   binary where the compiled AST must be linked to the host program at host program compile time.
    #   ie: the result of it's compilation can NOT be shared with another binary
    #
    # - Note that the latter case (compiling without using an existing mapping) results in portable
    #   code that can easily be re-located, address-wise, by offsetting all non-negative, non-zero
    #   addresses which are not jump operands of a zeroing operation (because these could be
    #   constants)
    #
    # - The difference is somewhat semantically similar to the difference between 'source'ing a file
    #   in bash, vs. executing a script.
    #
    # - Portable compilation is appropriate for 'fork-and-exec' style thread semantics, where the
    #   loaded program replaces the original, as well as other situations.
    
    # So our compilation workflow thus far looks like this:
    #
    #  |      
    #  |     (Build Macros and Strip from Source)
    #  |     (Expand Macros)   # same mechanism as 'load_from_source' or 'include' (the former rewrites variables
    #  |                       # so that the "compiled" source that is included has variables re-written, while
    #  |                       # the latter does not rewrite anything or imply scope - similar to 'source' in bash)
    #  |     (Tokenize Source)
    #  |     (Expand Includes)   # works the same way as macro, with *no* variable re-writing - this is a convenience
    #  |                         # for the user, ie: so they can have multiple files for organization - there are no
    #  |                         # new semantics implied here   # @TODO MAKE THIS WORK, IT'S UN-IMPLEMENTED RIGHT NOW!
    #  |
    #  |     (Compile)
    #  |         (Link external "libraries")   # portable code (ie: vars re-written), linked in via offset
    #  |
    #  V   
    #
    # The end of this flow produces a Klang 'binary' - ie: some form of a sequence of integers that can be executed
    # by the Klang vm starting at index 0 - this is a Klang vm 'kernel'.
    
    return stripped_program

    
test_source = """

MACRO test1
  some_macro = body
more = macro body
END MACRO testing A_LABEL:
        *aother = &macro body END
        
MACRO test3 one = two jump END

# SACRO test4 ANOTHER_LABEL:
#
#    HERE WTF am I doing with hygiene in macros - are we ditching it?
#
#         The SACRO thing is supposed to be for 'safe macro', so we
#         can choose between a macro that expands safely (rewritten
#         variables) or one which will capture identifiers (unhygienic).
#
#         But I think for bare-bones Klang, even this is too complicated.
#         USR registers are accessed directly in source, as appropriate,
#         and any named variables get 'allocated' as unused addresses in
#         the compiled source. It is simply understood that "all
#         variables are global", and we should not use any non-register
#         variables in macros that are supposed to be thread-safe or
#         isolated (ie: non-capturing/non-closing)?????
    
_OH_HAI_:
    a , b , c ,
    d , e , f ,   # comment
    g , h   # another comment
    
HAI_AGAIN:

    # comment
    
    a = 1
    a += 2
    a -= 3
    
    b = 0 HAI_AGAIN
    b += 2       _FIN_
    b -= 3   HAI_AGAIN   # never executed
    
    c = &a   # c is the address of a (ie: a USR address)
    d = *c   # d = 0 (value of a)
    *c = 1   # a is now 1
    
_FIN_:
    testing
    testblah	= world  # comment\nhello  =  again    \r\nhello =  again\r\n
"""

#print(test_source)   # Make sure the three-quote string is being parsed properly in python

pp(compile(test_source, GLOBAL_REGISTERS))

#assemble(test_source, None)

('\n'
 '\n'
 ' \n'
 '        \n'
 '\n'
 '\n'
 '# SACRO test4 ANOTHER_LABEL:\n'
 '#\n'
 '#    HERE WTF am I doing with hygiene in macros - are we ditching it?\n'
 '#\n'
 "#         The SACRO thing is supposed to be for 'safe macro', so we\n"
 '#         can choose between a macro that expands safely (rewritten\n'
 '#         variables) or one which will capture identifiers (unhygienic).\n'
 '#\n'
 '#         But I think for bare-bones Klang, even this is too complicated.\n'
 '#         USR registers are accessed directly in source, as appropriate,\n'
 "#         and any named variables get 'allocated' as unused addresses in\n"
 '#         the compiled source. It is simply understood that "all\n'
 '#         variables are global", and we should not use any non-register\n'
 '#         variables in macros that are supposed to be thread-safe or\n'
 '#         isolated (ie: non-capturing/non-closing)?????\n'
 '    \n'
 '_OH_HAI_:\n'
 '    a , b , c ,\n'
 '    d , e , f ,   # comment\n'
 '   

Now we have a parser/assembler/compiler that will take a limited form of human-readable source code (bare-bones Klang), and produce an executable binary kernel.

However, the source code syntax we have created, while Turing-complete, is still rather limited as the basis for a meta-language. For starters, we don't even have a way to define or call functions... instead we have an archaic form of GOTO, which while useful sometimes (especially at rather low levels of abstraction, like assembly), is not what we want to have to work with on a regular basis - instead, we want control structures that we can then represent via syntax, to encapsulate any common control-flow patterns that we might devise or want to otherwise include in our language.

We also have no easy way of creating variable-length values/structures of any kind, since the only paradigm approaching a container of some sort thus far is address-indexing to step through the individual addresses in a block (pointer arithmetic, etc - *See K&R, pp94 et al*).

Lastly, as we've already discussed briefly, at the high level that we tend to use in our heads as programmers, most all computation can be boiled down to lists of things - a program is often implied to be a list of instructions to carry out, functional programming is not sequential in terms of the program itself, although 'lists' of consecutive (parallel) function definitions abound, and calling functions, whether in the context of a functional language or otherwise, involves passing lists of parameters. Even currying, to allow single-argument functions to be composed into compound functions which effectively operate on multiple arguments to the whole abstraction, exists to emulate lists in languages/situations which otherwise explicitly disallow them. Lists are everywhere in programming in the context of data too - being able to only manipulate single values on their own would quickly become cumbersome, which is why we have arrays, tuples, dictionaries, sets, enumerations... the *list* goes on. Furthermore, when considering programming in the context of mathematics (a framework for defining and executing algorithms against parameters), there are countless well-known mathematical algorithms/procedures/etc which by definition operate over various forms of collecions of values, and not only single values (matrices in 3d graphics, for just one rather obvious example). So... in this context, it's obvious that any higher-level languages will almost always need some way to represent lists of things ("higher-order" data structures will always simply be various permutations and/or nestings of the basic list - as a minimalist reduction, any data object will always either be (or be able to be modelled as) a single thing, or a list of things (with the possibility for things in the list to themselves be lists).

So this now boils down to a few fresh minimum requirements for a *useful* and *practical* high-level meta-language:

* We need some way of representing a list or a container
* We need some way of defining a function (a 'thing' which can accept input and (optionally?) produce output)
* We need some way of calling functions (and capturing/otherwise being able to reference their result(s))

Let's start with the first one. To be useful, what does a generic container need?

* a way to reference it's member elements
* a way to enumerate the container
* since elements can be either individual values, or more containers, a way for a container to indicate the type of it's member elements

Ideally, we also need our container representation to be backwards-compatible with our existing data model, namely that an address' value can either be used as a value or the address of another value (value vs. pointer).

Since we already have and use 'pointers', let's leverage that and have the data structure for a container be a collection of pointers to it's elements. If we use a linked list, and let the 0 address indicate that there are no more elements in the list, then we get enumeration for free:

```
   +--------+
01 |     03 |   # This member value is at address 03 (value of this member is 123) 
   +--------+
02 |     05 |   # Next member *entry* is at address 05
   +--------+
03 |    123 |   # Value for first element
   +--------+
04 |    456 |   # Value for second element
   +--------+
05 |     04 |   # This member value is at address 04 (value of this member is 456)
   +--------+
06 |      0 |   # End of list - no more members
   +--------+
   ...
   +--------+
90 |     01 |   # This is the 'variable' pointing to the (beginning) entry in the linked list
   +--------+
```

Right now, the list itself occupies two addresses per child element, plus the space for the children themselves. Note that the children, if raw values, don't necessarily need the usual two addresses per variable that regular variables do - the first 'half' of the current structure for each entry in the list (the address of the value) can be though of as the address that the compiler uses to hold a 'nornal' variable's address, ie: that normal second address that the compiler uses to 'point' a variable at an address holding a value is replaced by the first element in the child entry of the list.

We still don't know if a member is itself a list, or a 'raw' address/value. If we extend this structure by one address per entry, we can do the following:

```
   +--------+
01 |     04 | ----+   # This member value is at address 04 (value of this member is 123) 
   +--------+     |
02 |     00 |     |   # This member is a raw value (type 0)
   +--------+     |
03 |     06 | --+ |   # Next member *entry* is at address 06
   +--------+   | |
04 |    123 | <---+   # Value for first element - a raw value
   +--------+   | 
05 |     80 | <-|-+   # Value for second element - the address of a list
   +--------+   | |
   |        | ----+
   |        |   |
06 |     05 | <-+     # This member value is at address 05 (value of this member is 80)
   |        |
   +--------+
06 |     01 |         # This member is a list (type 1)
   +--------+         
07 |      0 | --x     # End of list - no more members
   +--------+
   ...
   +--------+
80 |   1356 |   # This is the first address in the structure for a different list
   +--------+
   ...
   +--------+
90 |     01 |   # This is the 'variable' pointing to the (beginning) entry in the linked list
   +--------+
```

Now we know if a list's members are raw values or lists themselves, and as such we can work with heterogeneous data, ie: both 'raw', single values, as well as any kind of a list.

Let's start writing our meta-language kernel (ie: the extra bits that we want in our high-level meta language, but which our turing-complete bare-bones Klang does not yet provide)... first, we can use our linked list data model to give ourselves some subroutines that perform common list-manipulating operations:

* to create an empty list, we point the variable at 0

* to create a list element (assume space already allocated/available), we simply write a trio of consecutive addresses: address of value, 0 (default type), 0 (default next element)

* to append an element entry to the end of a list, we update it's 2nd value (if necessary for type), and update the third value (next element) of the current last element to point to our new entry instead of 0

* to un-shift from the head, we point the new element entry's "next element" value at the current head, and point the variable itself at the new entry

* pop and shift are essentially the opposite of append (push) and unshift, respectively.

* if a address being treated as a list is 0, then it is empty (ie: is_empty(<list>))

* 'head' is just a dereference of the list itself (first address in 'structure' is a pointer to the value

* 'tail' is just a dereference of var+2, ie: "tail = *(list_var + 2)... **since the zero address always contains a zero, if a list var (result of tail, etc) is zero, then it's empty**

###
### HERE - How much of this linked-list structure do we want hard-coded into the assembly level, vs. allowing it to be
###        "pluggable"??? The shape of this basic structure seems to restrict the resulting implementations, which we
###        want to avoid if we can.
###



So we know *how* to make some functions/subroutines that will operate on lists... but how do we call functions, or define them (or 'subroutines' or 'procedures' if we have/want different semantics)? What about variable scope, something we haven't yet had to deal with at the assembly level - when a function references a free variable, what happens? What about closures? And how do registers work into this? (Remember, our parser/compiler is currently translating all variable names into register references.)

Consider this pseudo-code:

```
var func

###
### START FUNCTION DEFINITION
###
var pre_func_ns = save_namespace()

var a
var b

ARGUMENT_REST = 1   # an index, -1 by default (it is reset upon FUNCTION_END)
func = FUNCTION_START
   return a / call('sum', b)
FUNCTION_END
FUNCTION_NAMESPACE = save_namespace()   # Should support recursion if the name is bound in the namespace!
                                        # Also, closures!

restore_namespace(pre_func_ns)
###
### END FUNCTION DEFINITION
###

<some more code here until we want to call "func"... />

call('func', [1, 2, 3, 4, 5, 6, 7])   # NB that we're passing args as an array!

    ###
    # The kernel now does this when it sees the above call of "func"...
    ###

    restore_namespace(FUNCTION_NAMESPACE(func))
    next = top_of_namespace
    if ARGUMENT_REST >= 0:
        while ARGUMENT_REST != 0:
            *next = ARGS.pop()
            next = get_next_var_in_namespace(next)
            ARGUMENT_REST --
    *next = ARGS
    execute_body(FUNCTION_BODY(func))
```

But what is a namespace? What does that mean in the context of our current setup? Not much, but we can change that... up until now, the idea of a 'variable' with a name exists solely in the compiler - the compiled code contains no form of the string "test" when we compile a program with a variable having that name. Instead, we have a value that is stored somewhere, and a compiler that knows the address of the value with that name, which it also stores as a value (so a variable is currently just two memory locations - one, whose value is the address of the second, and the second whose value is the value being stored).

We can re-use some of the same simple ideas we used when constructing our linked list model to address this, and produce a compiled program which also stores the names of variables, etc:

```
           <prior entries>
                 |
    | ...    |   |
    +--------+   |
101 |    900 | <-+   # The address of the variable's value (101 is the 'variable address')
    +--------+
102 |    000 |       # The type of this variable (type 0 is an integer)
    +--------+
103 |    200 |       # The name of this variable (points to a string-list)
    +--------+
104 |    105 | --+   # The address of the next variable in the namespace
    +--------+   |
105 |    907 | <-+   # The address of the variable's value (105 is the 'variable address')
    +--------+
106 |    000 |       # The type of this variable (type 0 is an integer)
    +--------+
107 |    250 |       # The name of this variable (points to a string-list)
    +--------+
108 |    000 |       # END OF NAMESPACE (0 == no more variables)
    +--------+
    
```

Now, in order to lookup a variable, instead of relying on the compiler to keep track of all the addresses it needs, we simply build upon this new type of linked list that represents our namespace. By walking this new list, and comparing the value of the third element (it's name value, a string-list), we can find the variable we want and then lookup it's value.

In order to walk a tree, we need some new operations:

* array_compare() - to start with two arrays (ie: two strings) and compare them element-wise until we find a non-match (or else they are equivalent).

